In [0]:
## Marketplace Feature Table -- Summary
# Tools Used - Pyspark

#step-1) Load the user data and convert signup Date into Date format "yyyy-mm-dd"
#step-2) Load the visitor log data and preprocess to convert the dat into date format and convert all strings to lower case.
# step-3) Dropping columns
#  a) dropped webclient ID, city and country as they add no useful information.
#  b) Dropped all rows where there is no UserID. Since this information is may be of just visitors and we dont need this data.
# step-4) Imputing null values of visitor log data.
#  a) First fill the visit date column with most visited date for each user. The logic behind filling the null values of this column is that         the null values may belong to date that the user was most active on.
#   b) Fill the product ID column based on UserID and visit date. We find the most viewed product for each visit date and for each user and we       fill the null values with that product. The logic is for that visit date the user might be viewing the same product which is viewed the        most on that day. If there are still null values left then we fill the null valueswith top viewed product for each user.
#   c) Fill the activity column based on userID and product ID. We find the top most activity for each product id for each user. So we fill         the null values with that activity if there is product ID already present for that row for each user. If there are still null values           where there is no product ID present for that row that we fill those cells with top most activity for that user.
#step-5) There will be still some null values left because there may be Users who have no Visit date , product ID and activity present in the     cells. So these users might have not visited the site or there is no sufficient information for these users to fill the null values.
#   a) We fill the null values of the visit date column with top visited date for the entire table.
#   b) we fill the null values of productID column with top viewed product for the entire table.
#   c) we fill the null values of activity column with top most activity for entire table.

#Step-6) save the table for querying.

# step-7) Create input feature table .


In [0]:
from pyspark.sql.types import *

from pyspark.sql import functions as func
import re





df_user_data=spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("delimiter",",").load('/FileStore/tables/userTable.csv')



In [0]:
df_user_data.show(truncate = False)

+-------+--------------------------------+------------+
UserID |Signup Date |User Segment|
+-------+--------------------------------+------------+
U133159|2018-04-14 07:01:16.202607+00:00|C |
U129368|2017-12-02 09:38:41.584270+00:00|B |
U109654|2013-03-19 11:38:55+00:00 |B |
U108998|2018-01-18 08:29:51.627954+00:00|C |
U131393|2018-03-27 08:05:28.806800+00:00|B |
U101889|2018-01-24 12:50:00.225811+00:00|B |
U113233|2017-03-18 09:27:53.079335+00:00|B |
U115728|2017-08-18 11:40:29.685549+00:00|B |
U132899|2018-03-14 08:32:32.906136+00:00|B |
U107492|2018-04-10 20:42:50.794037+00:00|B |
U124573|2014-09-25 07:24:32+00:00 |B |
U121993|2018-03-30 05:49:26.933324+00:00|B |
U125731|2017-12-22 16:57:31.586742+00:00|B |
U102932|2018-02-10 20:30:29.736785+00:00|C |
U114104|2018-04-01 03:31:46.712373+00:00|C |
U115212|2016-03-10 04:56:16.580571+00:00|B |
U127832|2018-03-22 15:30:02.714319+00:00|B |
U129967|2018-02-17 04:20:02.421738+00:00|B |
U102610|2017-06-05 05:14:52.605598+00:00|A |
U112354|2017-09-02 12:47:49.103688+00:00|A |
+-------+--------------------------------+------------+
only showing top 20 rows

In [0]:
df_user_data.printSchema()

root
-- UserID: string (nullable = true)
-- Signup Date: string (nullable = true)
-- User Segment: string (nullable = true)

In [0]:
# convert Date to Date type. 

In [0]:
df_user_data = df_user_data.withColumn("Signup_Date",df_user_data['Signup Date'].cast(DateType()))

In [0]:
df_user_data=df_user_data.drop("Signup Date")

In [0]:
df_user_data.show(2)

+-------+------------+-----------+
 UserID|User Segment|Signup_Date|
+-------+------------+-----------+
U133159| C| 2018-04-14|
U129368| B| 2017-12-02|
+-------+------------+-----------+
only showing top 2 rows

In [0]:
df_user_data.printSchema()

root
-- UserID: string (nullable = true)
-- User Segment: string (nullable = true)
-- Signup_Date: date (nullable = true)

In [0]:
# Load the visitor log data

In [0]:
df_visitor_log=spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("delimiter",",").load('/FileStore/tables/VisitorLogsData.csv')

In [0]:
df_visitor_log.show(15,truncate = False)

+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
webClientID |VisitDateTime |ProductID|UserID |Activity|Browser |OS |City |Country |
+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
WI10000050298 |2018-05-07 04:28:45.970|pr100631 |null |null |Chrome Mobile|Android |Chennai |India |
WI10000025922 |2018-05-13 07:26:04.964|pr100707 |null |null |Chrome |Windows |null |Taiwan |
WI100000204522|2018-05-11 11:43:42.832|pr100030 |null |click |Chrome |windows |Gurgaon |India |
WI10000011974 |2018-05-13 15:20:23.436|Pr100192 |null |CLICK |Chrome |Windows | | |
WI100000441953|2018-05-08 20:44:25.238|Pr100762 |null |click |Chrome |mac os x|Iselin |United States |
WI10000042224 |2018-05-24 13:10:32.908|pr100109 |null |click |Chrome |Mac OS X|null |Taiwan |
WI100000532584|2018-05-23 14:52:18.564|pr100020 |null |click |Opera |Windows |Baku |Azerbaijan |
WI100000377250|2018-05-13 03:13:21.958|Pr101495 |null |CLICK |Chrome |Windows |Bayan Lepas|Malaysia |
WI10000031378 |2018-05-10 02:04:54.844|Pr100013 |null |click |Chrome |Windows |Chakwal |Pakistan |
WI100000278874|2018-05-25 13:40:03.094|Pr101701 |null |PAGELOAD|Chrome |Windows | | |
WI100000971872|2018-05-27 05:35:31.253|Pr100025 |null |null |Chrome Mobile|Android |null |India |
WI100000746503|2018-05-16 11:56:33.028|Pr100896 |null |pageload|Chrome |windows |Lugo |Italy |
WI100000535505|2018-05-23 18:40:27.934|Pr100086 |null |click |Chrome |Windows |null |Norway |
WI10000021996 |1527051855673000000 |Pr100526 |null |null |Chrome |Mac OS X|Escondido |United States Escondido|
WI100000112772|1526366895249000000 |Pr100017 |U106593|click |Chrome Mobile|Android | | |
+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
only showing top 15 rows

In [0]:
df_visitor_log.printSchema()

root
-- webClientID: string (nullable = true)
-- VisitDateTime: string (nullable = true)
-- ProductID: string (nullable = true)
-- UserID: string (nullable = true)
-- Activity: string (nullable = true)
-- Browser: string (nullable = true)
-- OS: string (nullable = true)
-- City: string (nullable = true)
-- Country: string (nullable = true)

In [0]:
# we drop all rows where there is no user ID in visitor log data

In [0]:
df_visitor_log=df_visitor_log.where(df_visitor_log["UserID"].isNotNull())

In [0]:
df_visitor_log.show(5)

+--------------+--------------------+---------+-------+--------+-------------+---------+-------------+-------------+
 webClientID| VisitDateTime|ProductID| UserID|Activity| Browser| OS| City| Country|
+--------------+--------------------+---------+-------+--------+-------------+---------+-------------+-------------+
WI100000112772| 1526366895249000000| Pr100017|U106593| click|Chrome Mobile| Android| | |
 WI1000009977| 1527058921790000000| Pr101008|U108297| null|Chrome Mobile| Android| Delhi| India|
WI100000223826|2018-05-10 06:28:...| Pr100241|U132443| null| Firefox| Windows| null| India|
 WI10000021998|2018-05-08 12:40:...| pr100495|U134616| click| Chrome| Windows|Cottage Grove|United States|
 WI10000036281|2018-05-11 15:35:...| Pr100363|U130784| click| Chrome|Chrome OS| New Delhi| India|
+--------------+--------------------+---------+-------+--------+-------------+---------+-------------+-------------+
only showing top 5 rows

In [0]:
#checking null values in visitor log table

In [0]:
df_visitor_log.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_visitor_log.columns]).show()

+-----------+-------------+---------+------+--------+-------+---+------+-------+
webClientID|VisitDateTime|ProductID|UserID|Activity|Browser| OS| City|Country|
+-----------+-------------+---------+------+--------+-------+---+------+-------+
 0| 64803| 51963| 0| 87757| 0| 0|236124| 42827|
+-----------+-------------+---------+------+--------+-------+---+------+-------+

In [0]:
# drop webclientID, city and country as I feel there is not much information that is useful from these columns

In [0]:
df_visitor_log=df_visitor_log.drop("webClientID","City","Country")

In [0]:
df_visitor_log.show(5)

+--------------------+---------+-------+--------+-------------+---------+
 VisitDateTime|ProductID| UserID|Activity| Browser| OS|
+--------------------+---------+-------+--------+-------------+---------+
 1526366895249000000| Pr100017|U106593| click|Chrome Mobile| Android|
 1527058921790000000| Pr101008|U108297| null|Chrome Mobile| Android|
2018-05-10 06:28:...| Pr100241|U132443| null| Firefox| Windows|
2018-05-08 12:40:...| pr100495|U134616| click| Chrome| Windows|
2018-05-11 15:35:...| Pr100363|U130784| click| Chrome|Chrome OS|
+--------------------+---------+-------+--------+-------------+---------+
only showing top 5 rows

In [0]:
# convert VisitDate to date type in visitor log

In [0]:
df_visitor_log= df_visitor_log.withColumn("VisitDate",func.when(df_visitor_log.VisitDateTime.rlike("^[0-9]*$"),func.to_timestamp(func.col("VisitDateTime")/1e9).cast(DateType())).otherwise(df_visitor_log['VisitDateTime'].cast(DateType())))

In [0]:
df_visitor_log.show()

+--------------------+---------+-------+--------+-------------+---------+----------+
 VisitDateTime|ProductID| UserID|Activity| Browser| OS| VisitDate|
+--------------------+---------+-------+--------+-------------+---------+----------+
 1526366895249000000| Pr100017|U106593| click|Chrome Mobile| Android|2018-05-15|
 1527058921790000000| Pr101008|U108297| null|Chrome Mobile| Android|2018-05-23|
2018-05-10 06:28:...| Pr100241|U132443| null| Firefox| Windows|2018-05-10|
2018-05-08 12:40:...| pr100495|U134616| click| Chrome| Windows|2018-05-08|
2018-05-11 15:35:...| Pr100363|U130784| click| Chrome|Chrome OS|2018-05-11|
2018-05-19 00:02:...| pr100340|U120983| click| Chrome| Windows|2018-05-19|
2018-05-19 04:51:...| Pr100166|U120287| click| Chrome| Windows|2018-05-19|
2018-05-07 05:54:...| pr101042|U124307| CLICK| Chrome| Mac OS X|2018-05-07|
2018-05-23 09:44:...| Pr101042|U113937| CLICK| Safari| mac os x|2018-05-23|
2018-05-13 13:17:...| Pr101042|U115735| click| Chrome| Windows|2018-05-13|
2018-05-16 12:39:...| Pr101964|U103943| click| Chrome| Windows|2018-05-16|
2018-05-14 06:37:...| Pr100573|U125369| click|Mobile Safari| iOS|2018-05-14|
2018-05-25 06:01:...| pr100044|U111160| click|Chrome Mobile| Android|2018-05-25|
2018-05-19 18:48:...| pr100152|U106132| click| Chrome| Mac OS X|2018-05-19|
2018-05-17 23:26:...| Pr101614|U123525|pageload|Chrome Mobile| Android|2018-05-17|
2018-05-21 05:28:...| Pr100339|U119431| click| Chrome| windows|2018-05-21|
2018-05-16 04:47:...| Pr100784|U101041| click| Chrome| Windows|2018-05-16|
2018-05-07 08:28:...| Pr101042|U101158| click| Chrome| Windows|2018-05-07|
 1526829725361000000| Pr100193|U120095| click| Chrome| Windows|2018-05-20|
2018-05-15 11:07:...| Pr100106|U110592| click| Chrome| Mac OS X|2018-05-15|
+--------------------+---------+-------+--------+-------------+---------+----------+
only showing top 20 rows

In [0]:
# Drop visitDateTime in visitor_log

In [0]:
#df_visitor_log=df_visitor_log.drop("VisitDateTime")

In [0]:
df_visitor_log.show(2)

+-------------------+---------+-------+--------+-------------+-------+----------+
 VisitDateTime|ProductID| UserID|Activity| Browser| OS| VisitDate|
+-------------------+---------+-------+--------+-------------+-------+----------+
1526366895249000000| Pr100017|U106593| click|Chrome Mobile|Android|2018-05-15|
1527058921790000000| Pr101008|U108297| null|Chrome Mobile|Android|2018-05-23|
+-------------------+---------+-------+--------+-------------+-------+----------+
only showing top 2 rows

In [0]:
# Recheck the null values count of visitdate

In [0]:
df_visitor_log.where(df_visitor_log["VisitDate"].isNull()).count()

Out[285]: 64803

In [0]:
# check if the unique UserID in user_data table and visitor_log table are same

In [0]:
df_user_data.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 34050|
+----------------------+

In [0]:
df_visitor_log.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 34050|
+----------------------+

In [0]:
# convert the text in all columns to lower case

In [0]:
df_visitor_log=df_visitor_log.select(func.lower(df_visitor_log["ProductID"]).alias("ProductID"),df_visitor_log["UserID"],func.lower(df_visitor_log["Activity"]).alias("Activity"),func.lower(df_visitor_log["Browser"]).alias("Browser"),func.lower(df_visitor_log["OS"]).alias("OS"),df_visitor_log["VisitDate"],df_visitor_log["VisitDateTime"])

In [0]:
df_visitor_log.show(10)

+---------+-------+--------+-------------+---------+----------+--------------------+
ProductID| UserID|Activity| Browser| OS| VisitDate| VisitDateTime|
+---------+-------+--------+-------------+---------+----------+--------------------+
 pr100017|U106593| click|chrome mobile| android|2018-05-15| 1526366895249000000|
 pr101008|U108297| null|chrome mobile| android|2018-05-23| 1527058921790000000|
 pr100241|U132443| null| firefox| windows|2018-05-10|2018-05-10 06:28:...|
 pr100495|U134616| click| chrome| windows|2018-05-08|2018-05-08 12:40:...|
 pr100363|U130784| click| chrome|chrome os|2018-05-11|2018-05-11 15:35:...|
 pr100340|U120983| click| chrome| windows|2018-05-19|2018-05-19 00:02:...|
 pr100166|U120287| click| chrome| windows|2018-05-19|2018-05-19 04:51:...|
 pr101042|U124307| click| chrome| mac os x|2018-05-07|2018-05-07 05:54:...|
 pr101042|U113937| click| safari| mac os x|2018-05-23|2018-05-23 09:44:...|
 pr101042|U115735| click| chrome| windows|2018-05-13|2018-05-13 13:17:...|
+---------+-------+--------+-------------+---------+----------+--------------------+
only showing top 10 rows

In [0]:
# Imputing the null values.
# 1) First we fill the null values of visit date column. Group by user ID and fill the top most visited day of each user for filling the null values of visit date for that user.

In [0]:
# convert df_visitor_log into table for sql queries

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
#Using Sql query find the top visited date for each user

In [0]:
df_visitdate_null=spark.sql("select UserID,VisitDate, count(VisitDate) as visit_count from df_visitor_log group by UserID,VisitDate order by visit_count desc")

In [0]:
df_visitdate_null.cache()

Out[297]: DataFrame[UserID: string, VisitDate: date, visit_count: bigint]

In [0]:
df_visitdate_null.show(5)

+-------+----------+-----------+
 UserID| VisitDate|visit_count|
+-------+----------+-----------+
U100347|2018-05-25| 1456|
U100347|2018-05-22| 1453|
U134751|2018-05-08| 1427|
U100347|2018-05-19| 1319|
U100347|2018-05-24| 1316|
+-------+----------+-----------+
only showing top 5 rows

In [0]:
# Remove all rows that contain null values as they should not be counted as top visited date

In [0]:
df_visitdate_null=df_visitdate_null.where(df_visitdate_null["VisitDate"].isNotNull())

In [0]:
df_visitdate_null.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 33357|
+----------------------+

In [0]:
#Extract only top visited date for each userID

In [0]:
df_visitdate_null_1=df_visitdate_null.groupBy("UserID").agg(func.first("VisitDate"), func.max("visit_count"))

In [0]:
df_visitdate_null_1.cache()

Out[304]: DataFrame[UserID: string, first(VisitDate): date, max(visit_count): bigint]

In [0]:
df_visitdate_null_1.show(5)

+-------+----------------+----------------+
 UserID|first(VisitDate)|max(visit_count)|
+-------+----------------+----------------+
U100499| 2018-05-27| 721|
U105775| 2018-05-07| 512|
U126884| 2018-05-14| 209|
U116749| 2018-05-21| 136|
U104238| 2018-05-26| 102|
+-------+----------------+----------------+
only showing top 5 rows

In [0]:
df_visitdate_null_1.count()

Out[306]: 33357

In [0]:
# fill the null values of visitdatetime in original table df_visitor_log with top visted dates for each user id

In [0]:
df_visitor_log=df_visitor_log.join(df_visitdate_null_1, on =["UserID"], how='left_outer')

In [0]:
df_visitor_log.show()

+-------+---------+--------+-------------+---------+----------+--------------------+----------------+----------------+
 UserID|ProductID|Activity| Browser| OS| VisitDate| VisitDateTime|first(VisitDate)|max(visit_count)|
+-------+---------+--------+-------------+---------+----------+--------------------+----------------+----------------+
U106593| pr100017| click|chrome mobile| android|2018-05-15| 1526366895249000000| 2018-05-15| 17|
U108297| pr101008| null|chrome mobile| android|2018-05-23| 1527058921790000000| 2018-05-08| 196|
U132443| pr100241| null| firefox| windows|2018-05-10|2018-05-10 06:28:...| 2018-05-10| 23|
U134616| pr100495| click| chrome| windows|2018-05-08|2018-05-08 12:40:...| 2018-05-08| 33|
U130784| pr100363| click| chrome|chrome os|2018-05-11|2018-05-11 15:35:...| 2018-05-11| 97|
U120983| pr100340| click| chrome| windows|2018-05-19|2018-05-19 00:02:...| 2018-05-18| 21|
U120287| pr100166| click| chrome| windows|2018-05-19|2018-05-19 04:51:...| 2018-05-20| 156|
U124307| pr101042| click| chrome| mac os x|2018-05-07|2018-05-07 05:54:...| 2018-05-07| 96|
U113937| pr101042| click| safari| mac os x|2018-05-23|2018-05-23 09:44:...| 2018-05-23| 616|
U115735| pr101042| click| chrome| windows|2018-05-13|2018-05-13 13:17:...| 2018-05-13| 200|
U103943| pr101964| click| chrome| windows|2018-05-16|2018-05-16 12:39:...| 2018-05-10| 65|
U125369| pr100573| click|mobile safari| ios|2018-05-14|2018-05-14 06:37:...| 2018-05-14| 11|
U111160| pr100044| click|chrome mobile| android|2018-05-25|2018-05-25 06:01:...| 2018-05-25| 40|
U106132| pr100152| click| chrome| mac os x|2018-05-19|2018-05-19 18:48:...| 2018-05-27| 33|
U123525| pr101614|pageload|chrome mobile| android|2018-05-17|2018-05-17 23:26:...| 2018-05-17| 3|
U119431| pr100339| click| chrome| windows|2018-05-21|2018-05-21 05:28:...| 2018-05-21| 50|
U101041| pr100784| click| chrome| windows|2018-05-16|2018-05-16 04:47:...| 2018-05-16| 82|
U101158| pr101042| click| chrome| windows|2018-05-07|2018-05-07 08:28:...| 2018-05-07| 292|
U120095| pr100193| click| chrome| windows|2018-05-20| 1526829725361000000| 2018-05-20| 19|
U110592| pr100106| click| chrome| mac os x|2018-05-15|2018-05-15 11:07:...| 2018-05-15| 16|
+-------+---------+--------+-------------+---------+----------+--------------------+----------------+----------------+
only showing top 20 rows

In [0]:
# lets check how many users has no visit date at all.

In [0]:
df_visitor_log.where(df_visitor_log["first(VisitDate)"].isNull()).select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 693|
+----------------------+

In [0]:
df_visitor_log.where(df_visitor_log["first(VisitDate)"].isNull()).count()

Out[312]: 805

In [0]:
# using coalesce fill the null values.

In [0]:
df_visitor_log=df_visitor_log.withColumn('visit_date', func.coalesce('VisitDate', 'first(VisitDate)'))

In [0]:
df_visitor_log=df_visitor_log.drop("VisitDate","first(VisitDate)","max(visit_count)")

In [0]:
df_visitor_log.cache()

Out[316]: DataFrame[UserID: string, ProductID: string, Activity: string, Browser: string, OS: string, VisitDateTime: string, visit_date: date]

In [0]:
df_visitor_log.show(5)

+-------+---------+--------+-------------+---------+--------------------+----------+
 UserID|ProductID|Activity| Browser| OS| VisitDateTime|visit_date|
+-------+---------+--------+-------------+---------+--------------------+----------+
U106593| pr100017| click|chrome mobile| android| 1526366895249000000|2018-05-15|
U108297| pr101008| null|chrome mobile| android| 1527058921790000000|2018-05-23|
U132443| pr100241| null| firefox| windows|2018-05-10 06:28:...|2018-05-10|
U134616| pr100495| click| chrome| windows|2018-05-08 12:40:...|2018-05-08|
U130784| pr100363| click| chrome|chrome os|2018-05-11 15:35:...|2018-05-11|
+-------+---------+--------+-------------+---------+--------------------+----------+
only showing top 5 rows

In [0]:
df_visitor_log.where(df_visitor_log["visit_date"].isNull()).count()

Out[318]: 805

In [0]:
df_visitor_log.show(5)

+-------+---------+--------+-------------+---------+--------------------+----------+
 UserID|ProductID|Activity| Browser| OS| VisitDateTime|visit_date|
+-------+---------+--------+-------------+---------+--------------------+----------+
U106593| pr100017| click|chrome mobile| android| 1526366895249000000|2018-05-15|
U108297| pr101008| null|chrome mobile| android| 1527058921790000000|2018-05-23|
U132443| pr100241| null| firefox| windows|2018-05-10 06:28:...|2018-05-10|
U134616| pr100495| click| chrome| windows|2018-05-08 12:40:...|2018-05-08|
U130784| pr100363| click| chrome|chrome os|2018-05-11 15:35:...|2018-05-11|
+-------+---------+--------+-------------+---------+--------------------+----------+
only showing top 5 rows

In [0]:
# There are still 805 rows of data in visitor log with null values in visit_date which we will address later.
#2) we will fill null values for productID column. we group by userID and visit_date to find the top viewed product for each user and for each visit_date. Suppose if the visit_date for any of the user has no productID viewed , then we take the top most product ID viewed by the user to fill null values.

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
df_product_null=spark.sql("select UserID,visit_date,productID, count(productID) as product_count from df_visitor_log group by UserID,visit_date,productID order by product_count desc")

In [0]:
df_product_null.cache()

Out[323]: DataFrame[UserID: string, visit_date: date, productID: string, product_count: bigint]

In [0]:
df_product_null.show()

+-------+----------+---------+-------------+
 UserID|visit_date|productID|product_count|
+-------+----------+---------+-------------+
U100347|2018-05-25| pr101042| 2368|
U134751|2018-05-08| pr100166| 1372|
U104056|2018-05-21| pr101042| 1220|
U100347|2018-05-22| pr101042| 1158|
U100347|2018-05-19| pr101042| 968|
U100499|2018-05-27| pr101042| 938|
U100881|2018-05-18| pr101042| 932|
U103200|2018-05-08| pr101042| 925|
U100347|2018-05-23| pr101042| 920|
U100347|2018-05-21| pr101042| 895|
U100503|2018-05-26| pr101042| 890|
U100347|2018-05-24| pr101042| 887|
U105324|2018-05-07| pr101042| 868|
U104165|2018-05-15| pr101042| 844|
U102614|2018-05-23| pr101042| 783|
U103161|2018-05-18| pr101042| 781|
U109780|2018-05-07| pr101042| 781|
U100347|2018-05-17| pr101042| 776|
U118598|2018-05-07| pr101042| 763|
U100267|2018-05-25| pr101042| 673|
+-------+----------+---------+-------------+
only showing top 20 rows

In [0]:
# remove the rows with null values

In [0]:
df_product_null=df_product_null.where(df_product_null["productID"].isNotNull())

In [0]:
df_product_null.show()

+-------+----------+---------+-------------+
 UserID|visit_date|productID|product_count|
+-------+----------+---------+-------------+
U100347|2018-05-25| pr101042| 2368|
U134751|2018-05-08| pr100166| 1372|
U104056|2018-05-21| pr101042| 1220|
U100347|2018-05-22| pr101042| 1158|
U100347|2018-05-19| pr101042| 968|
U100499|2018-05-27| pr101042| 938|
U100881|2018-05-18| pr101042| 932|
U103200|2018-05-08| pr101042| 925|
U100347|2018-05-23| pr101042| 920|
U100347|2018-05-21| pr101042| 895|
U100503|2018-05-26| pr101042| 890|
U100347|2018-05-24| pr101042| 887|
U105324|2018-05-07| pr101042| 868|
U104165|2018-05-15| pr101042| 844|
U102614|2018-05-23| pr101042| 783|
U103161|2018-05-18| pr101042| 781|
U109780|2018-05-07| pr101042| 781|
U100347|2018-05-17| pr101042| 776|
U118598|2018-05-07| pr101042| 763|
U100267|2018-05-25| pr101042| 673|
+-------+----------+---------+-------------+
only showing top 20 rows

In [0]:
# we extract the top viewed prodcut for each visit date for each user

In [0]:
df_product_null_1=df_product_null.groupBy("UserID","visit_date").agg(func.first("productID"), func.max("product_count"))

In [0]:
df_product_null_1.show()

+-------+----------+----------------+------------------+
 UserID|visit_date|first(productID)|max(product_count)|
+-------+----------+----------------+------------------+
U100002|2018-05-18| pr100258| 4|
U100003|2018-05-13| pr100178| 1|
U100004|2018-05-16| pr100355| 4|
U100004|2018-05-17| pr100753| 2|
U100004|2018-05-19| pr101448| 1|
U100005|2018-05-09| pr101814| 1|
U100005|2018-05-11| pr100812| 2|
U100006|2018-05-21| pr101111| 1|
U100007|2018-05-10| pr100209| 4|
U100008|2018-05-07| pr100444| 2|
U100008|2018-05-10| pr102009| 4|
U100008|2018-05-19| pr100822| 2|
U100008|2018-05-24| pr100184| 3|
U100008|2018-05-25| pr100184| 3|
U100008|2018-05-27| pr100962| 1|
U100009|2018-05-11| pr100259| 1|
U100009|2018-05-15| pr100167| 2|
U100009|2018-05-24| pr100000| 1|
U100012|2018-05-14| pr100055| 3|
U100013|2018-05-10| pr100481| 2|
+-------+----------+----------------+------------------+
only showing top 20 rows

In [0]:
df_product_null_1.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 33545|
+----------------------+

In [0]:
# fill the null values of productID in original table df_visitor_log with top productID for each user id and for each visit date

In [0]:
df_visitor_log=df_visitor_log.join(df_product_null_1, on =["UserID","visit_date"], how='left_outer')

In [0]:
df_visitor_log.show()

+-------+----------+---------+--------+-------------+---------+--------------------+----------------+------------------+
 UserID|visit_date|ProductID|Activity| Browser| OS| VisitDateTime|first(productID)|max(product_count)|
+-------+----------+---------+--------+-------------+---------+--------------------+----------------+------------------+
U106593|2018-05-15| pr100017| click|chrome mobile| android| 1526366895249000000| pr100017| 14|
U108297|2018-05-23| pr101008| null|chrome mobile| android| 1527058921790000000| pr101030| 5|
U132443|2018-05-10| pr100241| null| firefox| windows|2018-05-10 06:28:...| pr100241| 16|
U134616|2018-05-08| pr100495| click| chrome| windows|2018-05-08 12:40:...| pr100495| 34|
U130784|2018-05-11| pr100363| click| chrome|chrome os|2018-05-11 15:35:...| pr100363| 25|
U120983|2018-05-19| pr100340| click| chrome| windows|2018-05-19 00:02:...| pr100340| 14|
U120287|2018-05-19| pr100166| click| chrome| windows|2018-05-19 04:51:...| pr102152| 30|
U124307|2018-05-07| pr101042| click| chrome| mac os x|2018-05-07 05:54:...| pr101042| 36|
U113937|2018-05-23| pr101042| click| safari| mac os x|2018-05-23 09:44:...| pr101042| 628|
U115735|2018-05-13| pr101042| click| chrome| windows|2018-05-13 13:17:...| pr101042| 156|
U103943|2018-05-16| pr101964| click| chrome| windows|2018-05-16 12:39:...| pr101964| 30|
U125369|2018-05-14| pr100573| click|mobile safari| ios|2018-05-14 06:37:...| pr100573| 10|
U111160|2018-05-25| pr100044| click|chrome mobile| android|2018-05-25 06:01:...| pr102361| 9|
U106132|2018-05-19| pr100152| click| chrome| mac os x|2018-05-19 18:48:...| pr100152| 25|
U123525|2018-05-17| pr101614|pageload|chrome mobile| android|2018-05-17 23:26:...| pr101614| 3|
U119431|2018-05-21| pr100339| click| chrome| windows|2018-05-21 05:28:...| pr100910| 34|
U101041|2018-05-16| pr100784| click| chrome| windows|2018-05-16 04:47:...| pr100784| 86|
U101158|2018-05-07| pr101042| click| chrome| windows|2018-05-07 08:28:...| pr101042| 223|
U120095|2018-05-20| pr100193| click| chrome| windows| 1526829725361000000| pr100193| 18|
U110592|2018-05-15| pr100106| click| chrome| mac os x|2018-05-15 11:07:...| pr100106| 13|
+-------+----------+---------+--------+-------------+---------+--------------------+----------------+------------------+
only showing top 20 rows

In [0]:
# using colaese to fill the null values

In [0]:
df_visitor_log=df_visitor_log.withColumn('product_id', func.coalesce('ProductID', 'first(productID)'))

In [0]:
df_visitor_log=df_visitor_log.drop("ProductID","first(productID)","max(product_count)")

In [0]:
df_visitor_log.show()

+-------+----------+--------+-------------+---------+--------------------+----------+
 UserID|visit_date|Activity| Browser| OS| VisitDateTime|product_id|
+-------+----------+--------+-------------+---------+--------------------+----------+
U106593|2018-05-15| click|chrome mobile| android| 1526366895249000000| pr100017|
U108297|2018-05-23| null|chrome mobile| android| 1527058921790000000| pr101008|
U132443|2018-05-10| null| firefox| windows|2018-05-10 06:28:...| pr100241|
U134616|2018-05-08| click| chrome| windows|2018-05-08 12:40:...| pr100495|
U130784|2018-05-11| click| chrome|chrome os|2018-05-11 15:35:...| pr100363|
U120983|2018-05-19| click| chrome| windows|2018-05-19 00:02:...| pr100340|
U120287|2018-05-19| click| chrome| windows|2018-05-19 04:51:...| pr100166|
U124307|2018-05-07| click| chrome| mac os x|2018-05-07 05:54:...| pr101042|
U113937|2018-05-23| click| safari| mac os x|2018-05-23 09:44:...| pr101042|
U115735|2018-05-13| click| chrome| windows|2018-05-13 13:17:...| pr101042|
U103943|2018-05-16| click| chrome| windows|2018-05-16 12:39:...| pr101964|
U125369|2018-05-14| click|mobile safari| ios|2018-05-14 06:37:...| pr100573|
U111160|2018-05-25| click|chrome mobile| android|2018-05-25 06:01:...| pr100044|
U106132|2018-05-19| click| chrome| mac os x|2018-05-19 18:48:...| pr100152|
U123525|2018-05-17|pageload|chrome mobile| android|2018-05-17 23:26:...| pr101614|
U119431|2018-05-21| click| chrome| windows|2018-05-21 05:28:...| pr100339|
U101041|2018-05-16| click| chrome| windows|2018-05-16 04:47:...| pr100784|
U101158|2018-05-07| click| chrome| windows|2018-05-07 08:28:...| pr101042|
U120095|2018-05-20| click| chrome| windows| 1526829725361000000| pr100193|
U110592|2018-05-15| click| chrome| mac os x|2018-05-15 11:07:...| pr100106|
+-------+----------+--------+-------------+---------+--------------------+----------+
only showing top 20 rows

In [0]:
# For all the users that doesnot have prodcut ID for any of the visit date, fill the null values of remaining cells of productID column by top most product viewed by each user

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
# extract top viewed product for each user

In [0]:
df_product_null_2=spark.sql("select UserID,product_id, count(product_id) as max_count from df_visitor_log group by UserID, product_id order by max_count desc")

In [0]:
df_product_null_2.cache()

Out[343]: DataFrame[UserID: string, product_id: string, max_count: bigint]

In [0]:
df_product_null_2.show()

+-------+----------+---------+
 UserID|product_id|max_count|
+-------+----------+---------+
U100347| pr101042| 12782|
U100499| pr101042| 3828|
U103161| pr101042| 3815|
U100881| pr101042| 3037|
U104165| pr101042| 2945|
U103200| pr101042| 2656|
U100620| pr101042| 2315|
U103901| pr101042| 2097|
U104566| pr101042| 1787|
U102225| pr101042| 1712|
U103364| pr101042| 1707|
U134751| pr100166| 1509|
U104862| pr101042| 1483|
U104056| pr101042| 1469|
U103419| pr101042| 1375|
U105236| pr101042| 1346|
U102614| pr101042| 1343|
U102225| pr101367| 1255|
U101055| pr101042| 1175|
U104000| pr101042| 1151|
+-------+----------+---------+
only showing top 20 rows

In [0]:
df_product_null_2=df_product_null_2.where(df_product_null_2["product_id"].isNotNull())

In [0]:
df_product_null_2=df_product_null_2.groupBy("UserID").agg(func.first("product_id"), func.max("max_count"))

In [0]:
df_product_null_2.show()

+-------+-----------------+--------------+
 UserID|first(product_id)|max(max_count)|
+-------+-----------------+--------------+
U100002| pr100258| 4|
U100003| pr100079| 3|
U100004| pr100753| 7|
U100005| pr100234| 2|
U100006| pr101111| 1|
U100007| pr100209| 4|
U100008| pr100855| 18|
U100009| pr101070| 3|
U100012| pr100055| 29|
U100013| pr100177| 6|
U100014| pr100103| 7|
U100015| pr100390| 7|
U100016| pr100172| 8|
U100017| pr100119| 2|
U100018| pr100403| 33|
U100019| pr100017| 16|
U100020| pr100558| 10|
U100023| pr100181| 10|
U100024| pr100025| 3|
U100025| pr100240| 2|
+-------+-----------------+--------------+
only showing top 20 rows

In [0]:
# joining the dataframe creted with User ID and their top visited product with original table to fill the null values for each user

In [0]:
df_visitor_log=df_visitor_log.join(df_product_null_2, on =["UserID"], how='left_outer')

In [0]:
df_visitor_log.show()

+-------+----------+--------+-------------+---------+--------------------+----------+-----------------+--------------+
 UserID|visit_date|Activity| Browser| OS| VisitDateTime|product_id|first(product_id)|max(max_count)|
+-------+----------+--------+-------------+---------+--------------------+----------+-----------------+--------------+
U106593|2018-05-15| click|chrome mobile| android| 1526366895249000000| pr100017| pr100017| 24|
U108297|2018-05-23| null|chrome mobile| android| 1527058921790000000| pr101008| pr100651| 357|
U132443|2018-05-10| null| firefox| windows|2018-05-10 06:28:...| pr100241| pr100241| 16|
U134616|2018-05-08| click| chrome| windows|2018-05-08 12:40:...| pr100495| pr100495| 44|
U130784|2018-05-11| click| chrome|chrome os|2018-05-11 15:35:...| pr100363| pr100363| 39|
U120983|2018-05-19| click| chrome| windows|2018-05-19 00:02:...| pr100340| pr100340| 38|
U120287|2018-05-19| click| chrome| windows|2018-05-19 04:51:...| pr100166| pr100060| 187|
U124307|2018-05-07| click| chrome| mac os x|2018-05-07 05:54:...| pr101042| pr101042| 43|
U113937|2018-05-23| click| safari| mac os x|2018-05-23 09:44:...| pr101042| pr101042| 669|
U115735|2018-05-13| click| chrome| windows|2018-05-13 13:17:...| pr101042| pr101042| 302|
U103943|2018-05-16| click| chrome| windows|2018-05-16 12:39:...| pr101964| pr100413| 83|
U125369|2018-05-14| click|mobile safari| ios|2018-05-14 06:37:...| pr100573| pr100573| 10|
U111160|2018-05-25| click|chrome mobile| android|2018-05-25 06:01:...| pr100044| pr102361| 13|
U106132|2018-05-19| click| chrome| mac os x|2018-05-19 18:48:...| pr100152| pr100152| 26|
U123525|2018-05-17|pageload|chrome mobile| android|2018-05-17 23:26:...| pr101614| pr101614| 3|
U119431|2018-05-21| click| chrome| windows|2018-05-21 05:28:...| pr100339| pr100910| 40|
U101041|2018-05-16| click| chrome| windows|2018-05-16 04:47:...| pr100784| pr100784| 129|
U101158|2018-05-07| click| chrome| windows|2018-05-07 08:28:...| pr101042| pr101042| 265|
U120095|2018-05-20| click| chrome| windows| 1526829725361000000| pr100193| pr100193| 20|
U110592|2018-05-15| click| chrome| mac os x|2018-05-15 11:07:...| pr100106| pr100106| 18|
+-------+----------+--------+-------------+---------+--------------------+----------+-----------------+--------------+
only showing top 20 rows

In [0]:
# using coalaesce to fill the null values

In [0]:
df_visitor_log=df_visitor_log.withColumn('prod_ID', func.coalesce('product_id', 'first(product_id)'))

In [0]:
df_visitor_log=df_visitor_log.drop("product_id","first(product_id)","max(max_count)")

In [0]:
df_visitor_log.show()

+-------+----------+--------+-------------+---------+--------------------+--------+
 UserID|visit_date|Activity| Browser| OS| VisitDateTime| prod_ID|
+-------+----------+--------+-------------+---------+--------------------+--------+
U106593|2018-05-15| click|chrome mobile| android| 1526366895249000000|pr100017|
U108297|2018-05-23| null|chrome mobile| android| 1527058921790000000|pr101008|
U132443|2018-05-10| null| firefox| windows|2018-05-10 06:28:...|pr100241|
U134616|2018-05-08| click| chrome| windows|2018-05-08 12:40:...|pr100495|
U130784|2018-05-11| click| chrome|chrome os|2018-05-11 15:35:...|pr100363|
U120983|2018-05-19| click| chrome| windows|2018-05-19 00:02:...|pr100340|
U120287|2018-05-19| click| chrome| windows|2018-05-19 04:51:...|pr100166|
U124307|2018-05-07| click| chrome| mac os x|2018-05-07 05:54:...|pr101042|
U113937|2018-05-23| click| safari| mac os x|2018-05-23 09:44:...|pr101042|
U115735|2018-05-13| click| chrome| windows|2018-05-13 13:17:...|pr101042|
U103943|2018-05-16| click| chrome| windows|2018-05-16 12:39:...|pr101964|
U125369|2018-05-14| click|mobile safari| ios|2018-05-14 06:37:...|pr100573|
U111160|2018-05-25| click|chrome mobile| android|2018-05-25 06:01:...|pr100044|
U106132|2018-05-19| click| chrome| mac os x|2018-05-19 18:48:...|pr100152|
U123525|2018-05-17|pageload|chrome mobile| android|2018-05-17 23:26:...|pr101614|
U119431|2018-05-21| click| chrome| windows|2018-05-21 05:28:...|pr100339|
U101041|2018-05-16| click| chrome| windows|2018-05-16 04:47:...|pr100784|
U101158|2018-05-07| click| chrome| windows|2018-05-07 08:28:...|pr101042|
U120095|2018-05-20| click| chrome| windows| 1526829725361000000|pr100193|
U110592|2018-05-15| click| chrome| mac os x|2018-05-15 11:07:...|pr100106|
+-------+----------+--------+-------------+---------+--------------------+--------+
only showing top 20 rows

In [0]:
df_visitor_log.where(df_visitor_log["prod_id"].isNull()).count()

Out[355]: 582

In [0]:
# There are still 582 unique users with no product viewed which we will address later.
#2) we will fill null values for Activity column. we group by userID and product_id to find the top viewed activity for each user and for each product_id. Suppose if the product_id for any of the user has no Activity , then we take the top most activity for each user to fill null values.

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
df_activity_null=spark.sql("select UserID,prod_id,Activity, count(Activity) as activity_count from df_visitor_log group by UserID,prod_id,Activity order by activity_count desc")

In [0]:
df_activity_null.cache()

Out[359]: DataFrame[UserID: string, prod_id: string, Activity: string, activity_count: bigint]

In [0]:
df_activity_null.show()

+-------+--------+--------+--------------+
 UserID| prod_id|Activity|activity_count|
+-------+--------+--------+--------------+
U100347|pr101042| click| 10839|
U100499|pr101042| click| 3201|
U103161|pr101042| click| 3141|
U104165|pr101042| click| 2431|
U100881|pr101042| click| 2345|
U103200|pr101042| click| 2238|
U100620|pr101042| click| 1936|
U103901|pr101042| click| 1727|
U104566|pr101042| click| 1453|
U103364|pr101042| click| 1417|
U134751|pr100166|pageload| 1307|
U104056|pr101042| click| 1246|
U104862|pr101042| click| 1214|
U102225|pr101042| click| 1214|
U102614|pr101042| click| 1131|
U105236|pr101042| click| 1094|
U101055|pr101042| click| 964|
U104000|pr101042| click| 950|
U103419|pr101042| click| 937|
U100503|pr101042| click| 923|
+-------+--------+--------+--------------+
only showing top 20 rows

In [0]:
#Extract only rows where there are no null values for Activity

In [0]:
df_activity_null=df_activity_null.where(df_activity_null["Activity"].isNotNull())

In [0]:
df_activity_null.select(func.countDistinct('UserID')).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 33089|
+----------------------+

In [0]:
#find the top activity for each user and for each product

In [0]:
df_activity_null_1=df_activity_null.groupBy("UserID","prod_id").agg(func.first("Activity"), func.max("activity_count"))

In [0]:
df_activity_null_1.show()

+-------+--------+---------------+-------------------+
 UserID| prod_id|first(Activity)|max(activity_count)|
+-------+--------+---------------+-------------------+
U100002|pr100258| pageload| 2|
U100004|pr100142| pageload| 1|
U100004|pr100204| click| 3|
U100004|pr100312| click| 1|
U100004|pr100694| pageload| 1|
U100004|pr100753| pageload| 6|
U100004|pr101448| pageload| 1|
U100004|pr101505| pageload| 2|
U100005|pr100633| click| 1|
U100006|pr101111| pageload| 1|
U100007|pr100113| pageload| 1|
U100008|pr100010| pageload| 1|
U100008|pr100102| pageload| 2|
U100008|pr100182| pageload| 2|
U100008|pr100184| click| 5|
U100008|pr100560| pageload| 1|
U100008|pr100788| pageload| 1|
U100008|pr100822| pageload| 2|
U100008|pr100855| click| 9|
U100008|pr100962| pageload| 1|
+-------+--------+---------------+-------------------+
only showing top 20 rows

In [0]:
df_activity_null_1.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 33089|
+----------------------+

In [0]:
# fill the null values of Activity in original table df_visitor_log with top activity for each user id and for each product

In [0]:
df_visitor_log=df_visitor_log.join(df_activity_null_1, on =["UserID","prod_id"], how='left_outer')

In [0]:
df_visitor_log.show()

+-------+--------+----------+--------+-------------+---------+--------------------+---------------+-------------------+
 UserID| prod_ID|visit_date|Activity| Browser| OS| VisitDateTime|first(Activity)|max(activity_count)|
+-------+--------+----------+--------+-------------+---------+--------------------+---------------+-------------------+
U106593|pr100017|2018-05-15| click|chrome mobile| android| 1526366895249000000| click| 17|
U108297|pr101008|2018-05-23| null|chrome mobile| android| 1527058921790000000| pageload| 1|
U132443|pr100241|2018-05-10| null| firefox| windows|2018-05-10 06:28:...| click| 13|
U134616|pr100495|2018-05-08| click| chrome| windows|2018-05-08 12:40:...| click| 36|
U130784|pr100363|2018-05-11| click| chrome|chrome os|2018-05-11 15:35:...| click| 28|
U120983|pr100340|2018-05-19| click| chrome| windows|2018-05-19 00:02:...| click| 32|
U120287|pr100166|2018-05-19| click| chrome| windows|2018-05-19 04:51:...| click| 2|
U124307|pr101042|2018-05-07| click| chrome| mac os x|2018-05-07 05:54:...| click| 31|
U113937|pr101042|2018-05-23| click| safari| mac os x|2018-05-23 09:44:...| click| 562|
U115735|pr101042|2018-05-13| click| chrome| windows|2018-05-13 13:17:...| click| 214|
U103943|pr101964|2018-05-16| click| chrome| windows|2018-05-16 12:39:...| click| 44|
U125369|pr100573|2018-05-14| click|mobile safari| ios|2018-05-14 06:37:...| pageload| 6|
U111160|pr100044|2018-05-25| click|chrome mobile| android|2018-05-25 06:01:...| click| 6|
U106132|pr100152|2018-05-19| click| chrome| mac os x|2018-05-19 18:48:...| click| 21|
U123525|pr101614|2018-05-17|pageload|chrome mobile| android|2018-05-17 23:26:...| pageload| 3|
U119431|pr100339|2018-05-21| click| chrome| windows|2018-05-21 05:28:...| click| 24|
U101041|pr100784|2018-05-16| click| chrome| windows|2018-05-16 04:47:...| click| 98|
U101158|pr101042|2018-05-07| click| chrome| windows|2018-05-07 08:28:...| click| 221|
U120095|pr100193|2018-05-20| click| chrome| windows| 1526829725361000000| click| 16|
U110592|pr100106|2018-05-15| click| chrome| mac os x|2018-05-15 11:07:...| click| 13|
+-------+--------+----------+--------+-------------+---------+--------------------+---------------+-------------------+
only showing top 20 rows

In [0]:
# use coalesce to fill the null values

In [0]:
df_visitor_log=df_visitor_log.withColumn('Act', func.coalesce('Activity', 'first(activity)'))

In [0]:
df_visitor_log=df_visitor_log.drop("Activity","first(Activity)","max(activity_count)")

In [0]:
df_visitor_log.show(5)

+-------+--------+----------+-------------+---------+--------------------+--------+
 UserID| prod_ID|visit_date| Browser| OS| VisitDateTime| Act|
+-------+--------+----------+-------------+---------+--------------------+--------+
U106593|pr100017|2018-05-15|chrome mobile| android| 1526366895249000000| click|
U108297|pr101008|2018-05-23|chrome mobile| android| 1527058921790000000|pageload|
U132443|pr100241|2018-05-10| firefox| windows|2018-05-10 06:28:...| click|
U134616|pr100495|2018-05-08| chrome| windows|2018-05-08 12:40:...| click|
U130784|pr100363|2018-05-11| chrome|chrome os|2018-05-11 15:35:...| click|
+-------+--------+----------+-------------+---------+--------------------+--------+
only showing top 5 rows

In [0]:
df_visitor_log.where(df_visitor_log["Act"].isNull()).count()

Out[375]: 7694

In [0]:
# fill the remaining null values with top most activity for each user

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
df_activity_null_2=spark.sql("select UserID,Act, count(Act) as max_count from df_visitor_log group by UserID, Act order by max_count desc")

In [0]:
df_activity_null_2.cache()

Out[379]: DataFrame[UserID: string, Act: string, max_count: bigint]

In [0]:
df_activity_null_2.show()

+-------+--------+---------+
 UserID| Act|max_count|
+-------+--------+---------+
U100347| click| 13865|
U100499| click| 4390|
U103161| click| 3936|
U104165| click| 3330|
U100881| click| 3060|
U103200| click| 2862|
U100620| click| 2621|
U102225| click| 2312|
U103901| click| 2064|
U104566| click| 1827|
U103364| click| 1750|
U100881|pageload| 1689|
U105236| click| 1617|
U104056| click| 1610|
U104862| click| 1536|
U134751|pageload| 1516|
U100595| click| 1489|
U102614| click| 1488|
U103419| click| 1294|
U100766| click| 1219|
+-------+--------+---------+
only showing top 20 rows

In [0]:
df_activity_null_2=df_activity_null_2.where(df_activity_null_2["Act"].isNotNull())

In [0]:
df_activity_null_2.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 33089|
+----------------------+

In [0]:
df_activity_null_2=df_activity_null_2.groupBy("UserID").agg(func.first("Act"), func.max("max_count"))

In [0]:
df_activity_null_2.count()

Out[384]: 33089

In [0]:
df_visitor_log=df_visitor_log.join(df_activity_null_2,on =["UserID"], how='left_outer')

In [0]:
df_visitor_log.show()

+-------+--------+----------+-------------+---------+--------------------+--------+----------+--------------+
 UserID| prod_ID|visit_date| Browser| OS| VisitDateTime| Act|first(Act)|max(max_count)|
+-------+--------+----------+-------------+---------+--------------------+--------+----------+--------------+
U106593|pr100017|2018-05-15|chrome mobile| android| 1526366895249000000| click| click| 20|
U108297|pr101008|2018-05-23|chrome mobile| android| 1527058921790000000|pageload| click| 394|
U132443|pr100241|2018-05-10| firefox| windows|2018-05-10 06:28:...| click| click| 23|
U134616|pr100495|2018-05-08| chrome| windows|2018-05-08 12:40:...| click| click| 41|
U130784|pr100363|2018-05-11| chrome|chrome os|2018-05-11 15:35:...| click| click| 74|
U120983|pr100340|2018-05-19| chrome| windows|2018-05-19 00:02:...| click| click| 36|
U120287|pr100166|2018-05-19| chrome| windows|2018-05-19 04:51:...| click| click| 614|
U124307|pr101042|2018-05-07| chrome| mac os x|2018-05-07 05:54:...| click| click| 74|
U113937|pr101042|2018-05-23| safari| mac os x|2018-05-23 09:44:...| click| click| 686|
U115735|pr101042|2018-05-13| chrome| windows|2018-05-13 13:17:...| click| click| 493|
U103943|pr101964|2018-05-16| chrome| windows|2018-05-16 12:39:...| click| click| 183|
U125369|pr100573|2018-05-14|mobile safari| ios|2018-05-14 06:37:...| click| pageload| 17|
U111160|pr100044|2018-05-25|chrome mobile| android|2018-05-25 06:01:...| click| click| 28|
U106132|pr100152|2018-05-19| chrome| mac os x|2018-05-19 18:48:...| click| click| 49|
U123525|pr101614|2018-05-17|chrome mobile| android|2018-05-17 23:26:...|pageload| pageload| 4|
U119431|pr100339|2018-05-21| chrome| windows|2018-05-21 05:28:...| click| click| 116|
U101041|pr100784|2018-05-16| chrome| windows|2018-05-16 04:47:...| click| click| 127|
U101158|pr101042|2018-05-07| chrome| windows|2018-05-07 08:28:...| click| click| 330|
U120095|pr100193|2018-05-20| chrome| windows| 1526829725361000000| click| click| 53|
U110592|pr100106|2018-05-15| chrome| mac os x|2018-05-15 11:07:...| click| click| 38|
+-------+--------+----------+-------------+---------+--------------------+--------+----------+--------------+
only showing top 20 rows

In [0]:
#use coleasce to fill null values

In [0]:
df_visitor_log=df_visitor_log.withColumn('Activity', func.coalesce('Act', 'first(Act)'))

In [0]:
df_visitor_log=df_visitor_log.drop("Act","first(Act)","max(max_count)")

In [0]:
df_visitor_log.cache()

Out[390]: DataFrame[UserID: string, prod_ID: string, visit_date: date, Browser: string, OS: string, VisitDateTime: string, Activity: string]

In [0]:
df_visitor_log.show()

+-------+--------+----------+-------+--------+--------------------+--------+
 UserID| prod_ID|visit_date|Browser| OS| VisitDateTime|Activity|
+-------+--------+----------+-------+--------+--------------------+--------+
U100008|pr100102|2018-05-12| chrome| android|2018-05-12 20:39:...|pageload|
U100008|pr100102|2018-05-10| chrome| android|2018-05-10 00:40:...| click|
U100008|pr100102|2018-05-12| chrome| android|2018-05-12 19:42:...|pageload|
U100013|pr100177|2018-05-17| chrome|mac os x|2018-05-17 05:38:...| click|
U100013|pr100177|2018-05-26| chrome|mac os x|2018-05-26 19:31:...|pageload|
U100013|pr100177|2018-05-26| safari|mac os x|2018-05-26 20:14:...|pageload|
U100013|pr100177|2018-05-17| chrome|mac os x| 1526533875378000000|pageload|
U100013|pr100177|2018-05-26| safari|mac os x|2018-05-26 21:11:...| click|
U100013|pr100177|2018-05-26| safari|mac os x|2018-05-26 20:14:...|pageload|
U100027|pr100371|2018-05-07| chrome| windows|2018-05-07 13:51:...|pageload|
U100034|pr103235|2018-05-08| chrome| windows|2018-05-08 13:35:...| click|
U100034|pr103235|2018-05-08| chrome| windows|2018-05-08 13:26:...|pageload|
U100070|pr100153|2018-05-23| chrome| windows| null| click|
U100070|pr100153|2018-05-27| chrome| windows|2018-05-27 21:38:...| click|
U100070|pr100153|2018-05-27| chrome| windows|2018-05-27 21:38:...| click|
U100070|pr100153|2018-05-27| chrome| windows|2018-05-27 21:38:...| click|
U100070|pr100153|2018-05-27| chrome| windows|2018-05-27 21:38:...| click|
U100081|pr100390|2018-05-26| chrome| windows|2018-05-26 16:07:...|pageload|
U100081|pr100390|2018-05-26| chrome| windows|2018-05-26 16:07:...|pageload|
U100081|pr100390|2018-05-16| chrome| windows|2018-05-16 20:49:...| click|
+-------+--------+----------+-------+--------+--------------------+--------+
only showing top 20 rows

In [0]:
# Lets check how many null values left in the dataframe.

In [0]:
df_visitor_log.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_visitor_log.columns]).show()

+------+-------+----------+-------+---+-------------+--------+
UserID|prod_ID|visit_date|Browser| OS|VisitDateTime|Activity|
+------+-------+----------+-------+---+-------------+--------+
 0| 582| 805| 0| 0| 64803| 1146|
+------+-------+----------+-------+---+-------------+--------+

In [0]:
#df_visitor_log.write.csv('/FileStore/tables/visitor_log_intermediate.csv',header = 'true')

In [0]:
# there are still few null values in prod_id, visit_date and Activity column. we fill the null values with the corresponding top most appeared value for that column or simply mode of that column.

In [0]:
#df_visitor_log=spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("delimiter",",").load('/FileStore/tables/visitor_log_intermediate.csv')

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
spark.sql("select prod_id,count(prod_id) as count from df_visitor_log group by prod_id order by count desc limit 1").show()

+--------+-----+
 prod_id|count|
+--------+-----+
pr101042|89194|
+--------+-----+

In [0]:
spark.sql("select Activity,count(Activity) as count from df_visitor_log group by Activity order by count desc limit 1").show()

+--------+------+
Activity| count|
+--------+------+
 click|464973|
+--------+------+

In [0]:
spark.sql("select visit_date,count(visit_date) as count from df_visitor_log group by visit_date order by count desc limit 1").show()

+----------+-----+
visit_date|count|
+----------+-----+
2018-05-07|50033|
+----------+-----+

In [0]:
df_visitor_log=df_visitor_log.fillna( { 'prod_id':"pr101042", 'Activity':"click"} )

In [0]:
df_visitor_log=df_visitor_log.withColumn('visit_date_time',func.when(func.col('visit_date').isNull(),func.to_date(func.lit('07.05.2018'),'dd.MM.yyyy')).otherwise(func.col('visit_date')))

In [0]:
df_visitor_log.drop('visit_date')

Out[403]: DataFrame[UserID: string, prod_ID: string, Browser: string, OS: string, VisitDateTime: string, Activity: string, visit_date_time: date]

In [0]:
df_visitor_log.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_visitor_log.columns]).show()

+------+-------+----------+-------+---+-------------+--------+---------------+
UserID|prod_ID|visit_date|Browser| OS|VisitDateTime|Activity|visit_date_time|
+------+-------+----------+-------+---+-------------+--------+---------------+
 0| 0| 805| 0| 0| 64803| 0| 0|
+------+-------+----------+-------+---+-------------+--------+---------------+

In [0]:
df_visitor_log=df_visitor_log.drop("visit_date")

In [0]:
# Convert the VisitDateTime column as timestamp type. This column is required for querying since we need to hours and minutes information to order by date.

In [0]:
df_visitor_log= df_visitor_log.withColumn("date",func.when(df_visitor_log.VisitDateTime.rlike("^[0-9]*$"),func.to_timestamp(func.col("VisitDateTime")/1e9)).otherwise(df_visitor_log['VisitDateTime']))

In [0]:
df_visitor_log.printSchema()

root
-- UserID: string (nullable = true)
-- prod_ID: string (nullable = false)
-- Browser: string (nullable = true)
-- OS: string (nullable = true)
-- VisitDateTime: string (nullable = true)
-- Activity: string (nullable = false)
-- visit_date_time: date (nullable = true)
-- date: string (nullable = true)

In [0]:
df_visitor_log.registerTempTable("df_visitor_log")

In [0]:
df_visitor_log=spark.sql('select UserID,prod_ID,Activity,Browser,OS,visit_date_time,CAST(date AS TIMESTAMP) from df_visitor_log')

In [0]:
df_visitor_log= df_visitor_log.withColumn("temp",func.to_timestamp(func.col("visit_date_time")))

In [0]:
df_visitor_log.show()

+-------+--------+--------+-------------+---------+---------------+--------------------+-------------------+
 UserID| prod_ID|Activity| Browser| OS|visit_date_time| date| temp|
+-------+--------+--------+-------------+---------+---------------+--------------------+-------------------+
U106593|pr100017| click|chrome mobile| android| 2018-05-15|2018-05-15 06:48:...|2018-05-15 00:00:00|
U108297|pr101008|pageload|chrome mobile| android| 2018-05-23|2018-05-23 07:02:...|2018-05-23 00:00:00|
U132443|pr100241| click| firefox| windows| 2018-05-10|2018-05-10 06:28:...|2018-05-10 00:00:00|
U134616|pr100495| click| chrome| windows| 2018-05-08|2018-05-08 12:40:...|2018-05-08 00:00:00|
U130784|pr100363| click| chrome|chrome os| 2018-05-11|2018-05-11 15:35:...|2018-05-11 00:00:00|
U120983|pr100340| click| chrome| windows| 2018-05-19|2018-05-19 00:02:...|2018-05-19 00:00:00|
U120287|pr100166| click| chrome| windows| 2018-05-19|2018-05-19 04:51:...|2018-05-19 00:00:00|
U124307|pr101042| click| chrome| mac os x| 2018-05-07|2018-05-07 05:54:...|2018-05-07 00:00:00|
U113937|pr101042| click| safari| mac os x| 2018-05-23|2018-05-23 09:44:...|2018-05-23 00:00:00|
U115735|pr101042| click| chrome| windows| 2018-05-13|2018-05-13 13:17:...|2018-05-13 00:00:00|
U103943|pr101964| click| chrome| windows| 2018-05-16|2018-05-16 12:39:...|2018-05-16 00:00:00|
U125369|pr100573| click|mobile safari| ios| 2018-05-14|2018-05-14 06:37:...|2018-05-14 00:00:00|
U111160|pr100044| click|chrome mobile| android| 2018-05-25|2018-05-25 06:01:...|2018-05-25 00:00:00|
U106132|pr100152| click| chrome| mac os x| 2018-05-19|2018-05-19 18:48:...|2018-05-19 00:00:00|
U123525|pr101614|pageload|chrome mobile| android| 2018-05-17|2018-05-17 23:26:...|2018-05-17 00:00:00|
U119431|pr100339| click| chrome| windows| 2018-05-21|2018-05-21 05:28:...|2018-05-21 00:00:00|
U101041|pr100784| click| chrome| windows| 2018-05-16|2018-05-16 04:47:...|2018-05-16 00:00:00|
U101158|pr101042| click| chrome| windows| 2018-05-07|2018-05-07 08:28:...|2018-05-07 00:00:00|
U120095|pr100193| click| chrome| windows| 2018-05-20|2018-05-20 15:22:...|2018-05-20 00:00:00|
U110592|pr100106| click| chrome| mac os x| 2018-05-15|2018-05-15 11:07:...|2018-05-15 00:00:00|
+-------+--------+--------+-------------+---------+---------------+--------------------+-------------------+
only showing top 20 rows

In [0]:
df_visitor_log=df_visitor_log.withColumn('date_timestamp', func.coalesce('date', 'temp'))

In [0]:
df_visitor_log=df_visitor_log.drop("date","temp")

In [0]:
df_visitor_log.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_visitor_log.columns]).show()

+------+-------+--------+-------+---+---------------+--------------+
UserID|prod_ID|Activity|Browser| OS|visit_date_time|date_timestamp|
+------+-------+--------+-------+---+---------------+--------------+
 0| 0| 0| 0| 0| 0| 0|
+------+-------+--------+-------+---+---------------+--------------+

In [0]:
df_visitor_log.write.csv('/FileStore/tables/visitor_log_final1.csv',header = 'true')

In [0]:
# Save as text file to store the preprocessed data andfor further querying the data

In [0]:
df_visitor_data=spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").option("delimiter",",").load('/FileStore/tables/visitor_log_final1.csv')

In [0]:
#df_visitor_data=df_visitor_log

In [0]:
display(df_visitor_data)

UserID,prod_ID,Activity,Browser,OS,visit_date_time,date_timestamp
U106593,pr100017,click,chrome mobile,android,2018-05-15,2018-05-15T06:48:15.248+0000
U108297,pr101008,pageload,chrome mobile,android,2018-05-23,2018-05-23T07:02:01.790+0000
U132443,pr100241,click,firefox,windows,2018-05-10,2018-05-10T06:28:53.391+0000
U134616,pr100495,click,chrome,windows,2018-05-08,2018-05-08T12:40:02.153+0000
U130784,pr100363,click,chrome,chrome os,2018-05-11,2018-05-11T15:35:43.689+0000
U120983,pr100340,click,chrome,windows,2018-05-19,2018-05-19T00:02:31.347+0000
U120287,pr100166,click,chrome,windows,2018-05-19,2018-05-19T04:51:45.337+0000
U124307,pr101042,click,chrome,mac os x,2018-05-07,2018-05-07T05:54:39.408+0000
U113937,pr101042,click,safari,mac os x,2018-05-23,2018-05-23T09:44:44.023+0000
U115735,pr101042,click,chrome,windows,2018-05-13,2018-05-13T13:17:03.751+0000


In [0]:
# Extract the user vintage for each user

In [0]:
df_input_feature=df_user_data.withColumn("User_Vintage", 
              func.datediff(func.to_date(func.lit("2018-05-28")),
                       df_user_data["Signup_Date"]))

In [0]:
df_input_feature.show()

+-------+------------+-----------+------------+
 UserID|User Segment|Signup_Date|User_Vintage|
+-------+------------+-----------+------------+
U133159| C| 2018-04-14| 44|
U129368| B| 2017-12-02| 177|
U109654| B| 2013-03-19| 1896|
U108998| C| 2018-01-18| 130|
U131393| B| 2018-03-27| 62|
U101889| B| 2018-01-24| 124|
U113233| B| 2017-03-18| 436|
U115728| B| 2017-08-18| 283|
U132899| B| 2018-03-14| 75|
U107492| B| 2018-04-10| 48|
U124573| B| 2014-09-25| 1341|
U121993| B| 2018-03-30| 59|
U125731| B| 2017-12-22| 157|
U102932| C| 2018-02-10| 107|
U114104| C| 2018-04-01| 57|
U115212| B| 2016-03-10| 809|
U127832| B| 2018-03-22| 67|
U129967| B| 2018-02-17| 100|
U102610| A| 2017-06-05| 357|
U112354| A| 2017-09-02| 268|
+-------+------------+-----------+------------+
only showing top 20 rows

In [0]:
df_input_feature=df_input_feature.drop("User Segment","Signup_Date")

In [0]:
df_input_feature.show()

+-------+------------+
 UserID|User_Vintage|
+-------+------------+
U133159| 44|
U129368| 177|
U109654| 1896|
U108998| 130|
U131393| 62|
U101889| 124|
U113233| 436|
U115728| 283|
U132899| 75|
U107492| 48|
U124573| 1341|
U121993| 59|
U125731| 157|
U102932| 107|
U114104| 57|
U115212| 809|
U127832| 67|
U129967| 100|
U102610| 357|
U112354| 268|
+-------+------------+
only showing top 20 rows

In [0]:
## No of days user visited in last 7 days

In [0]:
df_visitor_data.registerTempTable("df_visitor_data")

In [0]:
# filter data where date is greater than 20-05-2018

In [0]:
df_no_of_days=df_visitor_data.filter(df_visitor_data["visit_date_time"]>func.to_date(func.lit("2018-05-20")))

In [0]:
df_no_of_days.cache()

Out[433]: DataFrame[UserID: string, prod_ID: string, Activity: string, Browser: string, OS: string, visit_date_time: string, date_timestamp: timestamp]

In [0]:
df_no_of_days.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 16172|
+----------------------+

In [0]:
# only 16172 users visited in last 7 days

In [0]:
df_no_of_days.registerTempTable("df_no_of_days")

In [0]:
df_no_of_days_visited=spark.sql("select UserID, count(Distinct(visit_date_time)) as no_of_days_visited_7_days from df_no_of_days group by UserID")

In [0]:
df_input_feature=df_input_feature.join(df_no_of_days_visited, on=["UserID"], how='left_outer')

In [0]:
# fill the cells of users who not visited in last 7 days as 0

In [0]:
df_input_feature=df_input_feature.fillna({'no_of_days_visited_7_days':0})

In [0]:
df_input_feature=df_input_feature.drop("User Segment","Signup_Date")

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|
+-------+------------+-------------------------+
U133159| 44| 0|
U129368| 177| 0|
U109654| 1896| 0|
U108998| 130| 1|
U131393| 62| 0|
U101889| 124| 0|
U113233| 436| 1|
U115728| 283| 1|
U132899| 75| 0|
U107492| 48| 1|
U124573| 1341| 0|
U121993| 59| 0|
U125731| 157| 0|
U102932| 107| 1|
U114104| 57| 0|
U115212| 809| 1|
U127832| 67| 0|
U129967| 100| 0|
U102610| 357| 2|
U112354| 268| 1|
+-------+------------+-------------------------+
only showing top 20 rows

In [0]:
# Number of products viewed in the last 15 days for each user

In [0]:
df_no_of_products=df_visitor_data.filter(df_visitor_data["visit_date_time"]>func.to_date(func.lit("2018-05-12")))

In [0]:
df_no_of_products.registerTempTable("df_no_of_products")

In [0]:
df_no_of_prodcuts_viewed=spark.sql("select UserID, count(Distinct(prod_id)) as no_of_products_viewed_15_days from df_no_of_products group by UserID")

In [0]:
df_no_of_prodcuts_viewed.show()

+-------+-----------------------------+
 UserID|no_of_products_viewed_15_days|
+-------+-----------------------------+
U121194| 3|
U117953| 6|
U128906| 1|
U105501| 5|
U100754| 8|
U105775| 13|
U107157| 14|
U100499| 50|
U117383| 4|
U116488| 3|
U105809| 2|
U103874| 6|
U123774| 2|
U117960| 1|
U116749| 12|
U121504| 5|
U102315| 3|
U117118| 3|
U117376| 4|
U112099| 7|
+-------+-----------------------------+
only showing top 20 rows

In [0]:
df_input_feature=df_input_feature.join(df_no_of_prodcuts_viewed, on=["UserID"], how='left_outer')

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|
+-------+------------+-------------------------+-----------------------------+
U133159| 44| 0| null|
U129368| 177| 0| 1|
U109654| 1896| 0| 2|
U108998| 130| 1| 1|
U131393| 62| 0| null|
U101889| 124| 0| 2|
U113233| 436| 1| 2|
U115728| 283| 1| 1|
U132899| 75| 0| null|
U107492| 48| 1| 5|
U124573| 1341| 0| 1|
U121993| 59| 0| 1|
U125731| 157| 0| 1|
U102932| 107| 1| 1|
U114104| 57| 0| null|
U115212| 809| 1| 3|
U127832| 67| 0| 1|
U129967| 100| 0| null|
U102610| 357| 2| 6|
U112354| 268| 1| 1|
+-------+------------+-------------------------+-----------------------------+
only showing top 20 rows

In [0]:
# As we can see some users have not viewed any products in the last 15 days and for those fill with 0

In [0]:
df_input_feature=df_input_feature.fillna({'no_of_products_viewed_15_days':0})

In [0]:
df_input_feature.cache()

Out[452]: DataFrame[UserID: string, User_Vintage: int, no_of_days_visited_7_days: bigint, no_of_products_viewed_15_days: bigint]

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|
+-------+------------+-------------------------+-----------------------------+
U100364| 309| 0| 1|
U100499| 474| 7| 50|
U100608| 453| 3| 3|
U100611| 697| 0| 1|
U100754| 642| 2| 8|
U100999| 1881| 1| 1|
U102315| 86| 6| 3|
U103221| 55| 0| 1|
U103279| 121| 0| 0|
U103647| 100| 5| 3|
U103859| 1095| 3| 4|
U103874| 879| 2| 6|
U103924| 234| 2| 2|
U104238| 424| 3| 8|
U104440| 116| 1| 1|
U104625| 656| 4| 7|
U104644| 60| 2| 2|
U104722| 65| 6| 1|
U105187| 99| 1| 2|
U105380| 919| 1| 2|
+-------+------------+-------------------------+-----------------------------+
only showing top 20 rows

In [0]:
# Find the top viewed product in the last 15 days for each user

In [0]:
# first arrange the count of products viewed in the last 15 days for each user in desc order

In [0]:
df_most_viewed_product=spark.sql("select UserID,prod_id as Most_Viewed_product_15_Days,Activity, count(Activity) as activity_count from df_no_of_products  where Activity == 'pageload' group by UserID,Most_Viewed_product_15_Days,Activity order by activity_count desc")

In [0]:
df_most_viewed_product.cache()

Out[457]: DataFrame[UserID: string, Most_Viewed_product_15_Days: string, Activity: string, activity_count: bigint]

In [0]:
df_most_viewed_product.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 22973|
+----------------------+

In [0]:
df_most_viewed_product.show(20)

+-------+---------------------------+--------+--------------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count|
+-------+---------------------------+--------+--------------+
U100881| pr101367|pageload| 904|
U102225| pr101367|pageload| 353|
U100881| pr101042|pageload| 250|
U103419| pr101042|pageload| 209|
U102225| pr101042|pageload| 207|
U100881| pr100051|pageload| 195|
U100347| pr101042|pageload| 186|
U100347| pr101367|pageload| 174|
U102227| pr100166|pageload| 148|
U100881| pr102463|pageload| 141|
U100620| pr101367|pageload| 138|
U103161| pr101367|pageload| 127|
U103901| pr101367|pageload| 126|
U103161| pr101042|pageload| 120|
U103419| pr101367|pageload| 116|
U104165| pr101367|pageload| 107|
U100799| pr100051|pageload| 105|
U100347| pr100051|pageload| 105|
U100620| pr101042|pageload| 101|
U104862| pr101367|pageload| 89|
+-------+---------------------------+--------+--------------+
only showing top 20 rows

In [0]:
# second extract the visit date for each user and for each product

In [0]:
df_most_viewed_product2=spark.sql("select UserID,prod_ID as Most_Viewed_product_15_Days,date_timestamp from df_no_of_products where Activity=='pageload' group by UserID,Most_Viewed_product_15_Days,date_timestamp order by UserID")

In [0]:
df_most_viewed_product2.cache()

Out[467]: DataFrame[UserID: string, Most_Viewed_product_15_Days: string, date_timestamp: timestamp]

In [0]:
df_most_viewed_product2.show(50)

+-------+---------------------------+--------------------+
 UserID|Most_Viewed_product_15_Days| date_timestamp|
+-------+---------------------------+--------------------+
U100002| pr100258|2018-05-18 10:43:...|
U100002| pr100258|2018-05-18 10:39:...|
U100002| pr100258|2018-05-18 10:35:...|
U100002| pr101097|2018-05-18 10:39:...|
U100003| pr100079|2018-05-25 02:15:...|
U100004| pr100753|2018-05-18 11:34:...|
U100004| pr100753|2018-05-17 12:42:...|
U100004| pr100753|2018-05-20 14:38:...|
U100004| pr100312|2018-05-15 13:41:...|
U100004| pr100197|2018-05-17 13:52:...|
U100004| pr101554|2018-05-13 22:04:...|
U100004| pr101448|2018-05-19 12:26:...|
U100004| pr100408|2018-05-17 11:40:...|
U100004| pr101505|2018-05-17 12:12:...|
U100004| pr100142|2018-05-16 18:25:...|
U100004| pr100753|2018-05-20 14:38:...|
U100004| pr100753|2018-05-21 13:36:...|
U100004| pr100069|2018-05-20 14:38:...|
U100004| pr100591|2018-05-17 12:44:...|
U100004| pr100753| 2018-05-17 00:00:00|
U100004| pr100311|2018-05-17 12:57:...|
U100004| pr100694|2018-05-13 22:03:...|
U100005| pr100234|2018-05-26 01:06:...|
U100005| pr100234|2018-05-26 01:06:...|
U100005| pr100430|2018-05-19 23:27:...|
U100005| pr102072|2018-05-14 21:50:...|
U100006| pr101111|2018-05-21 07:21:...|
U100008| pr100560|2018-05-26 10:55:...|
U100008| pr100184|2018-05-24 18:07:...|
U100008| pr100606|2018-05-26 11:01:...|
U100008| pr100822|2018-05-19 20:25:...|
U100008| pr100788|2018-05-21 11:43:...|
U100008| pr100444|2018-05-19 20:28:...|
U100008| pr100962|2018-05-27 02:14:...|
U100008| pr100184|2018-05-25 21:49:...|
U100008| pr100182|2018-05-26 17:03:...|
U100008| pr100498|2018-05-23 22:35:...|
U100008| pr100234|2018-05-26 11:01:...|
U100008| pr100855|2018-05-26 11:01:...|
U100008| pr100025|2018-05-26 21:23:...|
U100008| pr100855| 2018-05-26 00:00:00|
U100008| pr101686|2018-05-23 16:18:...|
U100008| pr100498|2018-05-23 22:36:...|
U100008| pr100855|2018-05-26 10:57:...|
U100008| pr100089|2018-05-26 18:36:...|
U100008| pr100182| 2018-05-26 00:00:00|
U100008| pr100184|2018-05-24 18:30:...|
U100008| pr100855|2018-05-26 10:55:...|
U100008| pr100498| 2018-05-26 00:00:00|
U100008| pr101287|2018-05-24 18:32:...|
+-------+---------------------------+--------------------+
only showing top 50 rows

In [0]:
#Extract the most recent date for each product and for each user

In [0]:
df_most_viewed_product2=df_most_viewed_product2.groupBy("UserID","Most_Viewed_product_15_Days").agg(func.max("date_timestamp").alias("visit_time_date"))

In [0]:
df_most_viewed_product2.show(50)

+-------+---------------------------+--------------------+
 UserID|Most_Viewed_product_15_Days| visit_time_date|
+-------+---------------------------+--------------------+
U100002| pr100258|2018-05-18 10:43:...|
U100002| pr101097|2018-05-18 10:39:...|
U100003| pr100079|2018-05-25 02:15:...|
U100004| pr100753|2018-05-21 13:36:...|
U100004| pr100312|2018-05-15 13:41:...|
U100004| pr100197|2018-05-17 13:52:...|
U100004| pr101554|2018-05-13 22:04:...|
U100004| pr101448|2018-05-19 12:26:...|
U100004| pr100408|2018-05-17 11:40:...|
U100004| pr101505|2018-05-17 12:12:...|
U100004| pr100142|2018-05-16 18:25:...|
U100004| pr100069|2018-05-20 14:38:...|
U100004| pr100591|2018-05-17 12:44:...|
U100004| pr100311|2018-05-17 12:57:...|
U100004| pr100694|2018-05-13 22:03:...|
U100005| pr100234|2018-05-26 01:06:...|
U100005| pr100430|2018-05-19 23:27:...|
U100005| pr102072|2018-05-14 21:50:...|
U100006| pr101111|2018-05-21 07:21:...|
U100008| pr100560|2018-05-26 10:55:...|
U100008| pr100184|2018-05-25 21:49:...|
U100008| pr100606|2018-05-26 11:01:...|
U100008| pr100822|2018-05-19 20:25:...|
U100008| pr100788|2018-05-21 11:43:...|
U100008| pr100444|2018-05-19 20:28:...|
U100008| pr100962|2018-05-27 02:14:...|
U100008| pr100182|2018-05-26 17:03:...|
U100008| pr100498| 2018-05-26 00:00:00|
U100008| pr100234|2018-05-26 11:01:...|
U100008| pr100855|2018-05-26 11:01:...|
U100008| pr100025|2018-05-26 21:23:...|
U100008| pr101686|2018-05-23 16:18:...|
U100008| pr100089|2018-05-26 18:36:...|
U100008| pr101287|2018-05-24 18:32:...|
U100008| pr100010|2018-05-24 09:56:...|
U100009| pr100209| 2018-05-15 00:00:00|
U100009| pr100086| 2018-05-15 00:00:00|
U100009| pr100017|2018-05-20 16:16:...|
U100009| pr102091|2018-05-22 21:35:...|
U100009| pr100021|2018-05-26 07:24:...|
U100009| pr100166|2018-05-18 21:04:...|
U100009| pr101070|2018-05-23 21:25:...|
U100009| pr100640|2018-05-26 12:00:...|
U100009| pr100233|2018-05-15 22:06:...|
U100009| pr101657| 2018-05-15 00:00:00|
U100009| pr100167|2018-05-15 22:02:...|
U100012| pr100017|2018-05-27 22:18:...|
U100012| pr100055|2018-05-27 22:19:...|
U100012| pr100415|2018-05-27 12:25:...|
U100012| pr100593|2018-05-14 20:30:...|
+-------+---------------------------+--------------------+
only showing top 50 rows

In [0]:
# Join the most recent date with most viewed product table

In [0]:
df_most_viewed_product3=df_most_viewed_product.join(df_most_viewed_product2, on=["UserID","Most_Viewed_product_15_Days"], how="left_outer")

In [0]:
df_most_viewed_product3.sort("UserID").show(50)

+-------+---------------------------+--------+--------------+--------------------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count| visit_time_date|
+-------+---------------------------+--------+--------------+--------------------+
U100002| pr100258|pageload| 3|2018-05-18 10:43:...|
U100002| pr101097|pageload| 1|2018-05-18 10:39:...|
U100003| pr100079|pageload| 1|2018-05-25 02:15:...|
U100004| pr100694|pageload| 1|2018-05-13 22:03:...|
U100004| pr100408|pageload| 1|2018-05-17 11:40:...|
U100004| pr100069|pageload| 1|2018-05-20 14:38:...|
U100004| pr100591|pageload| 1|2018-05-17 12:44:...|
U100004| pr100197|pageload| 1|2018-05-17 13:52:...|
U100004| pr100142|pageload| 1|2018-05-16 18:25:...|
U100004| pr101505|pageload| 2|2018-05-17 12:12:...|
U100004| pr100312|pageload| 1|2018-05-15 13:41:...|
U100004| pr100311|pageload| 1|2018-05-17 12:57:...|
U100004| pr101448|pageload| 1|2018-05-19 12:26:...|
U100004| pr100753|pageload| 7|2018-05-21 13:36:...|
U100004| pr101554|pageload| 1|2018-05-13 22:04:...|
U100005| pr100430|pageload| 1|2018-05-19 23:27:...|
U100005| pr100234|pageload| 2|2018-05-26 01:06:...|
U100005| pr102072|pageload| 1|2018-05-14 21:50:...|
U100006| pr101111|pageload| 1|2018-05-21 07:21:...|
U100008| pr100560|pageload| 1|2018-05-26 10:55:...|
U100008| pr100182|pageload| 2|2018-05-26 17:03:...|
U100008| pr100010|pageload| 1|2018-05-24 09:56:...|
U100008| pr100025|pageload| 1|2018-05-26 21:23:...|
U100008| pr100498|pageload| 3| 2018-05-26 00:00:00|
U100008| pr100089|pageload| 2|2018-05-26 18:36:...|
U100008| pr100234|pageload| 1|2018-05-26 11:01:...|
U100008| pr101686|pageload| 1|2018-05-23 16:18:...|
U100008| pr100788|pageload| 1|2018-05-21 11:43:...|
U100008| pr100444|pageload| 1|2018-05-19 20:28:...|
U100008| pr101287|pageload| 1|2018-05-24 18:32:...|
U100008| pr100855|pageload| 5|2018-05-26 11:01:...|
U100008| pr100822|pageload| 2|2018-05-19 20:25:...|
U100008| pr100606|pageload| 2|2018-05-26 11:01:...|
U100008| pr100184|pageload| 4|2018-05-25 21:49:...|
U100008| pr100962|pageload| 1|2018-05-27 02:14:...|
U100009| pr100167|pageload| 2|2018-05-15 22:02:...|
U100009| pr100017|pageload| 2|2018-05-20 16:16:...|
U100009| pr101657|pageload| 1| 2018-05-15 00:00:00|
U100009| pr100021|pageload| 1|2018-05-26 07:24:...|
U100009| pr100209|pageload| 1| 2018-05-15 00:00:00|
U100009| pr100640|pageload| 1|2018-05-26 12:00:...|
U100009| pr101070|pageload| 1|2018-05-23 21:25:...|
U100009| pr100233|pageload| 1|2018-05-15 22:06:...|
U100009| pr100166|pageload| 1|2018-05-18 21:04:...|
U100009| pr100086|pageload| 2| 2018-05-15 00:00:00|
U100009| pr102091|pageload| 2|2018-05-22 21:35:...|
U100012| pr100055|pageload| 5|2018-05-27 22:19:...|
U100012| pr100017|pageload| 3|2018-05-27 22:18:...|
U100012| pr100036|pageload| 1|2018-05-14 20:30:...|
U100012| pr100593|pageload| 1|2018-05-14 20:30:...|
+-------+---------------------------+--------+--------------+--------------------+
only showing top 50 rows

In [0]:
df_most_viewed_product3.registerTempTable("df_most_viewed_product3")

In [0]:
# Extracting the top viewed product for each user ID

In [0]:
df_most_viewed_product4=spark.sql('select UserID,max(activity_count) as max_count from df_most_viewed_product3 group by UserID')

In [0]:
df_most_viewed_product4.cache()

Out[479]: DataFrame[UserID: string, max_count: bigint]

In [0]:
df_most_viewed_product4.sort("UserID").show(50)

+-------+---------+
 UserID|max_count|
+-------+---------+
U100002| 3|
U100003| 1|
U100004| 7|
U100005| 2|
U100006| 1|
U100008| 5|
U100009| 2|
U100012| 5|
U100013| 4|
U100014| 2|
U100015| 2|
U100016| 1|
U100018| 6|
U100019| 2|
U100020| 2|
U100023| 10|
U100024| 2|
U100026| 2|
U100027| 8|
U100028| 2|
U100029| 1|
U100030| 1|
U100031| 3|
U100032| 3|
U100034| 16|
U100036| 1|
U100038| 8|
U100039| 1|
U100040| 3|
U100043| 5|
U100045| 2|
U100048| 2|
U100049| 2|
U100050| 18|
U100051| 2|
U100052| 2|
U100056| 1|
U100057| 1|
U100058| 18|
U100061| 1|
U100063| 3|
U100065| 2|
U100066| 2|
U100067| 4|
U100068| 1|
U100069| 3|
U100070| 32|
U100071| 2|
U100072| 1|
U100073| 2|
+-------+---------+
only showing top 50 rows

In [0]:
# we will join this to previous table to filter the most viewed products for each user and we may see User has multiple products which have siminumber of page loads.And for these users we filter based on most recent Date.

In [0]:
df_most_viewed_product5=df_most_viewed_product3.join(df_most_viewed_product4, on =["UserID"], how='left_outer')

In [0]:
df_most_viewed_product5.sort('UserID').show(50)

+-------+---------------------------+--------+--------------+--------------------+---------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count| visit_time_date|max_count|
+-------+---------------------------+--------+--------------+--------------------+---------+
U100002| pr101097|pageload| 1|2018-05-18 10:39:...| 3|
U100002| pr100258|pageload| 3|2018-05-18 10:43:...| 3|
U100003| pr100079|pageload| 1|2018-05-25 02:15:...| 1|
U100004| pr100694|pageload| 1|2018-05-13 22:03:...| 7|
U100004| pr100069|pageload| 1|2018-05-20 14:38:...| 7|
U100004| pr101448|pageload| 1|2018-05-19 12:26:...| 7|
U100004| pr100312|pageload| 1|2018-05-15 13:41:...| 7|
U100004| pr100591|pageload| 1|2018-05-17 12:44:...| 7|
U100004| pr100408|pageload| 1|2018-05-17 11:40:...| 7|
U100004| pr101505|pageload| 2|2018-05-17 12:12:...| 7|
U100004| pr100142|pageload| 1|2018-05-16 18:25:...| 7|
U100004| pr100197|pageload| 1|2018-05-17 13:52:...| 7|
U100004| pr100753|pageload| 7|2018-05-21 13:36:...| 7|
U100004| pr101554|pageload| 1|2018-05-13 22:04:...| 7|
U100004| pr100311|pageload| 1|2018-05-17 12:57:...| 7|
U100005| pr100430|pageload| 1|2018-05-19 23:27:...| 2|
U100005| pr100234|pageload| 2|2018-05-26 01:06:...| 2|
U100005| pr102072|pageload| 1|2018-05-14 21:50:...| 2|
U100006| pr101111|pageload| 1|2018-05-21 07:21:...| 1|
U100008| pr100855|pageload| 5|2018-05-26 11:01:...| 5|
U100008| pr101686|pageload| 1|2018-05-23 16:18:...| 5|
U100008| pr100010|pageload| 1|2018-05-24 09:56:...| 5|
U100008| pr100962|pageload| 1|2018-05-27 02:14:...| 5|
U100008| pr100560|pageload| 1|2018-05-26 10:55:...| 5|
U100008| pr101287|pageload| 1|2018-05-24 18:32:...| 5|
U100008| pr100182|pageload| 2|2018-05-26 17:03:...| 5|
U100008| pr100606|pageload| 2|2018-05-26 11:01:...| 5|
U100008| pr100025|pageload| 1|2018-05-26 21:23:...| 5|
U100008| pr100184|pageload| 4|2018-05-25 21:49:...| 5|
U100008| pr100444|pageload| 1|2018-05-19 20:28:...| 5|
U100008| pr100089|pageload| 2|2018-05-26 18:36:...| 5|
U100008| pr100822|pageload| 2|2018-05-19 20:25:...| 5|
U100008| pr100498|pageload| 3| 2018-05-26 00:00:00| 5|
U100008| pr100788|pageload| 1|2018-05-21 11:43:...| 5|
U100008| pr100234|pageload| 1|2018-05-26 11:01:...| 5|
U100009| pr101070|pageload| 1|2018-05-23 21:25:...| 2|
U100009| pr100017|pageload| 2|2018-05-20 16:16:...| 2|
U100009| pr100640|pageload| 1|2018-05-26 12:00:...| 2|
U100009| pr100021|pageload| 1|2018-05-26 07:24:...| 2|
U100009| pr100209|pageload| 1| 2018-05-15 00:00:00| 2|
U100009| pr100233|pageload| 1|2018-05-15 22:06:...| 2|
U100009| pr101657|pageload| 1| 2018-05-15 00:00:00| 2|
U100009| pr100167|pageload| 2|2018-05-15 22:02:...| 2|
U100009| pr100086|pageload| 2| 2018-05-15 00:00:00| 2|
U100009| pr100166|pageload| 1|2018-05-18 21:04:...| 2|
U100009| pr102091|pageload| 2|2018-05-22 21:35:...| 2|
U100012| pr100055|pageload| 5|2018-05-27 22:19:...| 5|
U100012| pr100415|pageload| 2|2018-05-27 12:25:...| 5|
U100012| pr100593|pageload| 1|2018-05-14 20:30:...| 5|
U100012| pr100036|pageload| 1|2018-05-14 20:30:...| 5|
+-------+---------------------------+--------+--------------+--------------------+---------+
only showing top 50 rows

In [0]:
df_most_viewed_product6=df_most_viewed_product5.filter(df_most_viewed_product5["activity_count"]==df_most_viewed_product5["max_count"])

In [0]:
df_most_viewed_product6.sort("UserID").show(20)

+-------+---------------------------+--------+--------------+--------------------+---------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count| visit_time_date|max_count|
+-------+---------------------------+--------+--------------+--------------------+---------+
U100002| pr100258|pageload| 3|2018-05-18 10:43:...| 3|
U100003| pr100079|pageload| 1|2018-05-25 02:15:...| 1|
U100004| pr100753|pageload| 7|2018-05-21 13:36:...| 7|
U100005| pr100234|pageload| 2|2018-05-26 01:06:...| 2|
U100006| pr101111|pageload| 1|2018-05-21 07:21:...| 1|
U100008| pr100855|pageload| 5|2018-05-26 11:01:...| 5|
U100009| pr100167|pageload| 2|2018-05-15 22:02:...| 2|
U100009| pr100086|pageload| 2| 2018-05-15 00:00:00| 2|
U100009| pr100017|pageload| 2|2018-05-20 16:16:...| 2|
U100009| pr102091|pageload| 2|2018-05-22 21:35:...| 2|
U100012| pr100055|pageload| 5|2018-05-27 22:19:...| 5|
U100013| pr100177|pageload| 4|2018-05-26 20:14:...| 4|
U100014| pr100103|pageload| 2|2018-05-25 00:27:...| 2|
U100015| pr100044|pageload| 2|2018-05-24 17:45:...| 2|
U100016| pr100172|pageload| 1|2018-05-25 19:19:...| 1|
U100018| pr100403|pageload| 6|2018-05-26 15:24:...| 6|
U100019| pr100017|pageload| 2|2018-05-27 08:44:...| 2|
U100020| pr100017|pageload| 2|2018-05-15 14:43:...| 2|
U100020| pr100558|pageload| 2|2018-05-15 14:30:...| 2|
U100023| pr100181|pageload| 10|2018-05-27 16:16:...| 10|
+-------+---------------------------+--------+--------------+--------------------+---------+
only showing top 20 rows

In [0]:
df_most_viewed_product6.registerTempTable('df_most_viewed_product6')

In [0]:
# Arrange the table in desc order based on visit date so that we filter the first row for each user

In [0]:
#df_most_viewed_product6=spark.sql("select UserID,Most_Viewed_product_15_Days,activity_count,visit_time_date from df_most_viewed_product6 group by UserID,Most_Viewed_product_15_Days,activity_count,visit_time_date order by visit_time_date desc")

In [0]:
df_most_viewed_product6=df_most_viewed_product6.sort(func.desc("visit_time_date"))

In [0]:
df_most_viewed_product6.show(20)

+-------+---------------------------+--------+--------------+--------------------+---------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count| visit_time_date|max_count|
+-------+---------------------------+--------+--------------+--------------------+---------+
U104296| pr100390|pageload| 4|2018-05-27 23:55:...| 4|
U104309| pr100076|pageload| 1|2018-05-27 23:47:...| 1|
U104307| pr100401|pageload| 1|2018-05-27 23:46:...| 1|
U104312| pr100793|pageload| 2|2018-05-27 23:43:...| 2|
U104313| pr100956|pageload| 2|2018-05-27 23:42:...| 2|
U101751| pr100044|pageload| 20|2018-05-27 23:39:...| 20|
U104314| pr100772|pageload| 1|2018-05-27 23:38:...| 1|
U104315| pr100014|pageload| 1|2018-05-27 23:33:...| 1|
U104321| pr100263|pageload| 1|2018-05-27 23:28:...| 1|
U104185| pr100636|pageload| 1|2018-05-27 23:23:...| 1|
U104309| pr100491|pageload| 1|2018-05-27 23:21:...| 1|
U104328| pr100010|pageload| 1|2018-05-27 23:19:...| 1|
U104329| pr100873|pageload| 2|2018-05-27 23:15:...| 2|
U104330| pr100477|pageload| 1|2018-05-27 23:15:...| 1|
U100682| pr100470|pageload| 6|2018-05-27 23:07:...| 6|
U104331| pr100039|pageload| 2|2018-05-27 23:06:...| 2|
U104337| pr100020|pageload| 1|2018-05-27 23:02:...| 1|
U104209| pr100138|pageload| 5|2018-05-27 23:00:...| 5|
U104342| pr100192|pageload| 1|2018-05-27 22:56:...| 1|
U104343| pr101042|pageload| 2|2018-05-27 22:54:...| 2|
+-------+---------------------------+--------+--------------+--------------------+---------+
only showing top 20 rows

In [0]:
df_most_viewed_product6.filter(df_most_viewed_product6["UserID"]=="U100009").show()

+-------+---------------------------+--------+--------------+--------------------+---------+
 UserID|Most_Viewed_product_15_Days|Activity|activity_count| visit_time_date|max_count|
+-------+---------------------------+--------+--------------+--------------------+---------+
U100009| pr102091|pageload| 2|2018-05-22 21:35:...| 2|
U100009| pr100017|pageload| 2|2018-05-20 16:16:...| 2|
U100009| pr100167|pageload| 2|2018-05-15 22:02:...| 2|
U100009| pr100086|pageload| 2| 2018-05-15 00:00:00| 2|
+-------+---------------------------+--------+--------------+--------------------+---------+

In [0]:
df_most_viewed_product7=df_most_viewed_product6.groupBy("UserID").agg(func.first("Most_Viewed_product_15_Days"),func.max("visit_time_date"))

In [0]:
df_most_viewed_product7.sort("UserID").show(20)

+-------+----------------------------------+--------------------+
 UserID|first(Most_Viewed_product_15_Days)|max(visit_time_date)|
+-------+----------------------------------+--------------------+
U100002| pr100258|2018-05-18 10:43:...|
U100003| pr100079|2018-05-25 02:15:...|
U100004| pr100753|2018-05-21 13:36:...|
U100005| pr100234|2018-05-26 01:06:...|
U100006| pr101111|2018-05-21 07:21:...|
U100008| pr100855|2018-05-26 11:01:...|
U100009| pr102091|2018-05-22 21:35:...|
U100012| pr100055|2018-05-27 22:19:...|
U100013| pr100177|2018-05-26 20:14:...|
U100014| pr100103|2018-05-25 00:27:...|
U100015| pr100044|2018-05-24 17:45:...|
U100016| pr100172|2018-05-25 19:19:...|
U100018| pr100403|2018-05-26 15:24:...|
U100019| pr100017|2018-05-27 08:44:...|
U100020| pr100017|2018-05-15 14:43:...|
U100023| pr100181|2018-05-27 16:16:...|
U100024| pr100025|2018-05-27 09:00:...|
U100026| pr100153|2018-05-26 18:07:...|
U100027| pr102905|2018-05-17 13:52:...|
U100028| pr100137|2018-05-20 21:31:...|
+-------+----------------------------------+--------------------+
only showing top 20 rows

In [0]:
# Now we extracted the most viewed product for each user in the last 15 dfays

In [0]:
df_input_feature=df_input_feature.join(df_most_viewed_product7, on=["UserID"], how="left_outer")

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+----------------------------------+--------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|max(visit_time_date)|
+-------+------------+-------------------------+-----------------------------+----------------------------------+--------------------+
U100364| 309| 0| 1| null| null|
U100499| 474| 7| 50| pr101042|2018-05-27 18:52:...|
U100608| 453| 3| 3| pr100283|2018-05-21 17:39:...|
U100611| 697| 0| 1| pr100138|2018-05-18 16:07:...|
U100754| 642| 2| 8| pr100072|2018-05-24 19:14:...|
U100999| 1881| 1| 1| pr100017|2018-05-26 06:49:...|
U102315| 86| 6| 3| pr100027|2018-05-27 13:58:...|
U103221| 55| 0| 1| pr100483|2018-05-20 17:31:...|
U103279| 121| 0| 0| null| null|
U103647| 100| 5| 3| pr100390|2018-05-27 09:18:...|
U103859| 1095| 3| 4| pr100321|2018-05-22 16:15:...|
U103874| 879| 2| 6| pr101367|2018-05-27 09:55:...|
U103924| 234| 2| 2| pr100945|2018-05-21 17:06:...|
U104238| 424| 3| 8| pr101367|2018-05-27 12:29:...|
U104440| 116| 1| 1| null| null|
U104625| 656| 4| 7| pr100210|2018-05-27 18:20:...|
U104644| 60| 2| 2| pr100771|2018-05-27 17:13:...|
U104722| 65| 6| 1| pr100166|2018-05-27 17:30:...|
U105187| 99| 1| 2| pr100017|2018-05-27 14:36:...|
U105380| 919| 1| 2| pr101800|2018-05-27 13:47:...|
+-------+------------+-------------------------+-----------------------------+----------------------------------+--------------------+
only showing top 20 rows

In [0]:
# For the users with no product viewed fill with "Product101"

In [0]:
df_input_feature=df_input_feature.fillna({'first(Most_Viewed_product_15_Days)':"Product101"})

In [0]:
df_input_feature=df_input_feature.drop("max(visit_time_date)")

In [0]:
df_input_feature.cache()

Out[499]: DataFrame[UserID: string, User_Vintage: int, no_of_days_visited_7_days: bigint, no_of_products_viewed_15_days: bigint, first(Most_Viewed_product_15_Days): string]

In [0]:
# Find the most frequent used OS by the user

In [0]:
 df_active_os=spark.sql("select UserID, OS as Most_Active_OS, count(OS) as OS_count from df_visitor_data group by UserID,Most_Active_OS order by OS_count desc")

In [0]:
df_active_os.show()

+-------+--------------+--------+
 UserID|Most_Active_OS|OS_count|
+-------+--------------+--------+
U100347| mac os x| 8877|
U100347| windows| 5649|
U100499| windows| 4615|
U104165| mac os x| 3738|
U100881| windows| 3379|
U103200| windows| 3103|
U102225| ubuntu| 3066|
U100620| windows| 2999|
U103161| linux| 2756|
U103364| windows| 1813|
U105236| windows| 1778|
U104862| ubuntu| 1762|
U104566| windows| 1760|
U100595| windows| 1746|
U104056| windows| 1677|
U103419| ubuntu| 1671|
U134751| windows| 1565|
U103901| linux| 1557|
U103161| windows| 1527|
U102614| windows| 1521|
+-------+--------------+--------+
only showing top 20 rows

In [0]:
df_active_os.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 34050|
+----------------------+

In [0]:
df_active_os=df_active_os.groupBy("UserID").agg(func.first("Most_Active_OS"), func.max("OS_count"))

In [0]:
df_active_os.show()

+-------+---------------------+-------------+
 UserID|first(Most_Active_OS)|max(OS_count)|
+-------+---------------------+-------------+
U100002| android| 5|
U100003| windows| 4|
U100004| windows| 30|
U100005| android| 10|
U100006| android| 1|
U100007| windows| 8|
U100008| android| 76|
U100009| android| 21|
U100012| mac os x| 40|
U100013| mac os x| 20|
U100014| windows| 22|
U100015| android| 17|
U100016| android| 8|
U100017| windows| 2|
U100018| windows| 52|
U100019| windows| 32|
U100020| mac os x| 13|
U100023| windows| 17|
U100024| android| 3|
U100025| windows| 2|
+-------+---------------------+-------------+
only showing top 20 rows

In [0]:
df_input_feature=df_input_feature.join(df_active_os, on=["UserID"], how="left_outer")

In [0]:
df_input_feature.cache()

Out[507]: DataFrame[UserID: string, User_Vintage: int, no_of_days_visited_7_days: bigint, no_of_products_viewed_15_days: bigint, first(Most_Viewed_product_15_Days): string, first(Most_Active_OS): string, max(OS_count): bigint]

In [0]:
# Find the most recent viewed product by the user

In [0]:
df_recently_viewed=spark.sql("select UserID,date_timestamp,prod_id as Recently_Viewed_Product from df_visitor_data where Activity=='pageload' group by UserID,date_timestamp,Recently_Viewed_Product")

In [0]:
df_recently_viewed=df_recently_viewed.sort(func.desc("date_timestamp"))

In [0]:
df_recently_viewed=df_recently_viewed.groupBy("UserID").agg(func.first("Recently_viewed_product"), func.max("date_timestamp"))

In [0]:
df_recently_viewed.cache()

Out[513]: DataFrame[UserID: string, first(Recently_viewed_product): string, max(date_timestamp): timestamp]

In [0]:
df_recently_viewed.show()

+-------+------------------------------+--------------------+
 UserID|first(Recently_viewed_product)| max(date_timestamp)|
+-------+------------------------------+--------------------+
U100499| pr100051|2018-05-27 21:02:...|
U100608| pr100341|2018-05-27 14:24:...|
U100611| pr100138|2018-05-18 16:07:...|
U100754| pr100390|2018-05-25 17:37:...|
U100999| pr100017|2018-05-26 06:49:...|
U102315| pr100027|2018-05-27 13:58:...|
U103221| pr100483|2018-05-20 17:31:...|
U103279| pr100017| 2018-05-07 00:00:00|
U103647| pr100390|2018-05-27 09:18:...|
U103859| pr100252|2018-05-26 18:24:...|
U103874| pr101042|2018-05-27 09:55:...|
U103924| pr101039|2018-05-24 13:28:...|
U104238| pr101367|2018-05-27 12:29:...|
U104625| pr100210|2018-05-27 18:20:...|
U104644| pr101240|2018-05-27 18:04:...|
U104722| pr100166|2018-05-27 17:30:...|
U105187| pr100220|2018-05-27 14:38:...|
U105380| pr101800|2018-05-27 13:47:...|
U105483| pr100145|2018-05-27 13:07:...|
U105501| pr100267|2018-05-26 13:35:...|
+-------+------------------------------+--------------------+
only showing top 20 rows

In [0]:
df_recently_viewed.where(df_recently_viewed["first(Recently_viewed_product)"].isNull()).count()

Out[515]: 0

In [0]:
df_recently_viewed.select(func.countDistinct("UserID")).show()

+----------------------+
count(DISTINCT UserID)|
+----------------------+
 30331|
+----------------------+

In [0]:
df_input_feature=df_input_feature.join(df_recently_viewed, on=["UserID"], how="left_outer")

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|first(Most_Active_OS)|max(OS_count)|first(Recently_viewed_product)| max(date_timestamp)|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+
U100364| 309| 0| 1| Product101| windows| 2| null| null|
U100499| 474| 7| 50| pr101042| windows| 4615| pr100051|2018-05-27 21:02:...|
U100608| 453| 3| 3| pr100283| android| 8| pr100341|2018-05-27 14:24:...|
U100611| 697| 0| 1| pr100138| linux| 4| pr100138|2018-05-18 16:07:...|
U100754| 642| 2| 8| pr100072| windows| 90| pr100390|2018-05-25 17:37:...|
U100999| 1881| 1| 1| pr100017| android| 4| pr100017|2018-05-26 06:49:...|
U102315| 86| 6| 3| pr100027| windows| 75| pr100027|2018-05-27 13:58:...|
U103221| 55| 0| 1| pr100483| windows| 3| pr100483|2018-05-20 17:31:...|
U103279| 121| 0| 0| Product101| android| 1| pr100017| 2018-05-07 00:00:00|
U103647| 100| 5| 3| pr100390| windows| 92| pr100390|2018-05-27 09:18:...|
U103859| 1095| 3| 4| pr100321| windows| 21| pr100252|2018-05-26 18:24:...|
U103874| 879| 2| 6| pr101367| android| 74| pr101042|2018-05-27 09:55:...|
U103924| 234| 2| 2| pr100945| android| 3| pr101039|2018-05-24 13:28:...|
U104238| 424| 3| 8| pr101367| linux| 218| pr101367|2018-05-27 12:29:...|
U104440| 116| 1| 1| Product101| windows| 6| null| null|
U104625| 656| 4| 7| pr100210| windows| 15| pr100210|2018-05-27 18:20:...|
U104644| 60| 2| 2| pr100771| windows| 72| pr101240|2018-05-27 18:04:...|
U104722| 65| 6| 1| pr100166| windows| 56| pr100166|2018-05-27 17:30:...|
U105187| 99| 1| 2| pr100017| windows| 4| pr100220|2018-05-27 14:38:...|
U105380| 919| 1| 2| pr101800| windows| 4| pr101800|2018-05-27 13:47:...|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+
only showing top 20 rows

In [0]:
# For the users with no product viewed fill with "Product101"

In [0]:
df_input_feature=df_input_feature.fillna({"first(Recently_viewed_product)":"Product101"})

In [0]:
df_input_feature.cache()

Out[521]: DataFrame[UserID: string, User_Vintage: int, no_of_days_visited_7_days: bigint, no_of_products_viewed_15_days: bigint, first(Most_Viewed_product_15_Days): string, first(Most_Active_OS): string, max(OS_count): bigint, first(Recently_viewed_product): string, max(date_timestamp): timestamp]

In [0]:
# To find the no. of pageloads and no. of clicks in the last 7 days

In [0]:
df_activity_recent=df_visitor_data.filter(df_visitor_data["visit_date_time"]>func.to_date(func.lit("2018-05-20")))

In [0]:
df_activity_recent.registerTempTable("df_activity_recent")

In [0]:
#Extract the number of pageloads in the last 7 days for each user

In [0]:
df_pageload_recent=spark.sql("select UserID,Activity,count(Activity) as Pageloads_last_7_days from df_activity_recent where Activity=='pageload' group by UserID,Activity")

In [0]:
df_pageload_recent.show()

+-------+--------+---------------------+
 UserID|Activity|Pageloads_last_7_days|
+-------+--------+---------------------+
U100279|pageload| 14|
U103969|pageload| 4|
U105995|pageload| 9|
U100095|pageload| 12|
U109992|pageload| 9|
U102054|pageload| 8|
U107889|pageload| 33|
U119384|pageload| 10|
U113289|pageload| 1|
U100031|pageload| 9|
U111777|pageload| 4|
U107765|pageload| 3|
U111512|pageload| 4|
U113054|pageload| 4|
U109421|pageload| 7|
U114577|pageload| 15|
U108892|pageload| 20|
U106507|pageload| 1|
U115319|pageload| 18|
U102024|pageload| 8|
+-------+--------+---------------------+
only showing top 20 rows

In [0]:
df_pageload_recent.count()

Out[528]: 14225

In [0]:
df_pageload_recent=df_pageload_recent.drop("Activity")

In [0]:
# Extract the number of clicks in the last 7 days for each user

In [0]:
df_click_recent=spark.sql("select UserID,Activity,count(Activity) as Clicks_last_7_days from df_activity_recent where Activity=='click' group by UserID,Activity")

In [0]:
df_click_recent=df_click_recent.drop("Activity")

In [0]:
df_click_recent.count()

Out[533]: 11294

In [0]:
df_input_feature=df_input_feature.join(df_pageload_recent, on=["UserID"], how="left_outer")

In [0]:
df_input_feature=df_input_feature.join(df_click_recent, on=["UserID"], how='left_outer')

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+---------------------+------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|first(Most_Active_OS)|max(OS_count)|first(Recently_viewed_product)| max(date_timestamp)|Pageloads_last_7_days|Clicks_last_7_days|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+---------------------+------------------+
U100364| 309| 0| 1| Product101| windows| 2| Product101| null| null| null|
U100499| 474| 7| 50| pr101042| windows| 4615| pr100051|2018-05-27 21:02:...| 151| 3112|
U100608| 453| 3| 3| pr100283| android| 8| pr100341|2018-05-27 14:24:...| 4| 2|
U100611| 697| 0| 1| pr100138| linux| 4| pr100138|2018-05-18 16:07:...| null| null|
U100754| 642| 2| 8| pr100072| windows| 90| pr100390|2018-05-25 17:37:...| 8| 40|
U100999| 1881| 1| 1| pr100017| android| 4| pr100017|2018-05-26 06:49:...| 1| 2|
U102315| 86| 6| 3| pr100027| windows| 75| pr100027|2018-05-27 13:58:...| 11| 37|
U103221| 55| 0| 1| pr100483| windows| 3| pr100483|2018-05-20 17:31:...| null| null|
U103279| 121| 0| 0| Product101| android| 1| pr100017| 2018-05-07 00:00:00| null| null|
U103647| 100| 5| 3| pr100390| windows| 92| pr100390|2018-05-27 09:18:...| 26| 22|
U103859| 1095| 3| 4| pr100321| windows| 21| pr100252|2018-05-26 18:24:...| 4| 15|
U103874| 879| 2| 6| pr101367| android| 74| pr101042|2018-05-27 09:55:...| 45| 29|
U103924| 234| 2| 2| pr100945| android| 3| pr101039|2018-05-24 13:28:...| 3| null|
U104238| 424| 3| 8| pr101367| linux| 218| pr101367|2018-05-27 12:29:...| 17| 194|
U104440| 116| 1| 1| Product101| windows| 6| Product101| null| null| 6|
U104625| 656| 4| 7| pr100210| windows| 15| pr100210|2018-05-27 18:20:...| 6| 3|
U104644| 60| 2| 2| pr100771| windows| 72| pr101240|2018-05-27 18:04:...| 3| 67|
U104722| 65| 6| 1| pr100166| windows| 56| pr100166|2018-05-27 17:30:...| 15| null|
U105187| 99| 1| 2| pr100017| windows| 4| pr100220|2018-05-27 14:38:...| 3| 1|
U105380| 919| 1| 2| pr101800| windows| 4| pr101800|2018-05-27 13:47:...| 2| 2|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+-------------+------------------------------+--------------------+---------------------+------------------+
only showing top 20 rows

In [0]:
df_input_feature=df_input_feature.drop("max(OS_count)","max(visit_date_time)")

In [0]:
df_input_feature.show()

+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+--------------------+---------------------+------------------+
 UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|first(Most_Active_OS)|first(Recently_viewed_product)| max(date_timestamp)|Pageloads_last_7_days|Clicks_last_7_days|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+--------------------+---------------------+------------------+
U100364| 309| 0| 1| Product101| windows| Product101| null| null| null|
U100499| 474| 7| 50| pr101042| windows| pr100051|2018-05-27 21:02:...| 151| 3112|
U100608| 453| 3| 3| pr100283| android| pr100341|2018-05-27 14:24:...| 4| 2|
U100611| 697| 0| 1| pr100138| linux| pr100138|2018-05-18 16:07:...| null| null|
U100754| 642| 2| 8| pr100072| windows| pr100390|2018-05-25 17:37:...| 8| 40|
U100999| 1881| 1| 1| pr100017| android| pr100017|2018-05-26 06:49:...| 1| 2|
U102315| 86| 6| 3| pr100027| windows| pr100027|2018-05-27 13:58:...| 11| 37|
U103221| 55| 0| 1| pr100483| windows| pr100483|2018-05-20 17:31:...| null| null|
U103279| 121| 0| 0| Product101| android| pr100017| 2018-05-07 00:00:00| null| null|
U103647| 100| 5| 3| pr100390| windows| pr100390|2018-05-27 09:18:...| 26| 22|
U103859| 1095| 3| 4| pr100321| windows| pr100252|2018-05-26 18:24:...| 4| 15|
U103874| 879| 2| 6| pr101367| android| pr101042|2018-05-27 09:55:...| 45| 29|
U103924| 234| 2| 2| pr100945| android| pr101039|2018-05-24 13:28:...| 3| null|
U104238| 424| 3| 8| pr101367| linux| pr101367|2018-05-27 12:29:...| 17| 194|
U104440| 116| 1| 1| Product101| windows| Product101| null| null| 6|
U104625| 656| 4| 7| pr100210| windows| pr100210|2018-05-27 18:20:...| 6| 3|
U104644| 60| 2| 2| pr100771| windows| pr101240|2018-05-27 18:04:...| 3| 67|
U104722| 65| 6| 1| pr100166| windows| pr100166|2018-05-27 17:30:...| 15| null|
U105187| 99| 1| 2| pr100017| windows| pr100220|2018-05-27 14:38:...| 3| 1|
U105380| 919| 1| 2| pr101800| windows| pr101800|2018-05-27 13:47:...| 2| 2|
+-------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+--------------------+---------------------+------------------+
only showing top 20 rows

In [0]:
# chack if there are any null values in the table

In [0]:
df_input_feature.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_input_feature.columns]).show()

+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+-------------------+---------------------+------------------+
UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|first(Most_Active_OS)|first(Recently_viewed_product)|max(date_timestamp)|Pageloads_last_7_days|Clicks_last_7_days|
+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+-------------------+---------------------+------------------+
 0| 0| 0| 0| 0| 0| 0| 3719| 19825| 22756|
+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+-------------------+---------------------+------------------+

In [0]:
# Fill the null values

In [0]:
df_input_feature=df_input_feature.drop("max(date_timestamp)")

In [0]:
df_input_feature=df_input_feature.fillna({'Pageloads_last_7_days':0,'Clicks_last_7_days':0})

In [0]:
df_input_feature.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_input_feature.columns]).show()

+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+---------------------+------------------+
UserID|User_Vintage|no_of_days_visited_7_days|no_of_products_viewed_15_days|first(Most_Viewed_product_15_Days)|first(Most_Active_OS)|first(Recently_viewed_product)|Pageloads_last_7_days|Clicks_last_7_days|
+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+---------------------+------------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+------------+-------------------------+-----------------------------+----------------------------------+---------------------+------------------------------+---------------------+------------------+

In [0]:
df_input_feature=df_input_feature.select(df_input_feature["UserID"],df_input_feature["no_of_days_Visited_7_Days"].alias("No_of_days_Visited_7_Days"),df_input_feature["no_of_products_viewed_15_days"].alias("No_Of_Products_Viewed_15_Days"),df_input_feature["User_Vintage"].alias("User_Vintage"),df_input_feature["first(Most_Viewed_product_15_Days)"].alias("Most_Viewed_product_15_Days"),df_input_feature["first(Most_Active_OS)"].alias("Most_Active_OS"),df_input_feature["first(Recently_viewed_product)"].alias("Recently_Viewed_Product"),df_input_feature["Pageloads_last_7_days"],df_input_feature["Clicks_last_7_days"])

In [0]:
df_input_feature=df_input_feature.sort("UserID")

In [0]:
df_input_feature.show()

+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
 UserID|No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
U100002| 0| 2| 53| pr100258| android| pr100258| 0| 0|
U100003| 1| 2| 1021| pr100079| windows| pr100079| 1| 2|
U100004| 1| 15| 341| pr100753| windows| pr100753| 1| 0|
U100005| 1| 3| 681| pr100234| android| pr100234| 2| 0|
U100006| 1| 1| 55| pr101111| android| pr101111| 1| 0|
U100007| 0| 0| 460| Product101| windows| pr100265| 0| 0|
U100008| 6| 19| 395| pr100855| android| pr100962| 26| 34|
U100009| 4| 13| 78| pr102091| android| pr100640| 5| 4|
U100012| 2| 6| 124| pr100055| mac os x| pr100055| 8| 21|
U100013| 3| 3| 1687| pr100177| mac os x| pr100134| 7| 3|
U100014| 2| 4| 155| pr100103| windows| pr100103| 1| 2|
U100015| 1| 5| 659| pr100044| android| pr100095| 3| 14|
U100016| 1| 1| 114| pr100172| android| pr100172| 1| 0|
U100017| 1| 1| 45| Product101| windows| Product101| 0| 2|
U100018| 5| 3| 44| pr100403| windows| pr100403| 7| 34|
U100019| 3| 8| 317| pr100017| windows| pr100040| 7| 21|
U100020| 0| 2| 163| pr100017| mac os x| pr100017| 0| 0|
U100023| 5| 3| 437| pr100181| windows| pr100181| 10| 0|
U100024| 1| 1| 85| pr100025| android| pr100025| 2| 1|
U100025| 0| 0| 129| Product101| windows| pr100240| 0| 0|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
only showing top 20 rows

In [0]:
df_final=df_input_feature

In [0]:
display(df_final)

UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
U100002,0,2,53,pr100258,android,pr100258,0,0
U100003,1,2,1021,pr100079,windows,pr100079,1,2
U100004,1,15,341,pr100753,windows,pr100753,1,0
U100005,1,3,681,pr100234,android,pr100234,2,0
U100006,1,1,55,pr101111,android,pr101111,1,0
U100007,0,0,460,Product101,windows,pr100265,0,0
U100008,6,19,395,pr100855,android,pr100962,26,34
U100009,4,13,78,pr102091,android,pr100640,5,4
U100012,2,6,124,pr100055,mac os x,pr100055,8,21
U100013,3,3,1687,pr100177,mac os x,pr100134,7,3
